#### Paquetes

In [1]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Optional, Union
from datetime import datetime
import pyodbc

#### Variables Hardcodeadas

In [2]:
plantas = ["MALABRIGO", "CHIMBOTE", "SUPE", "VEGUETA", "CALLAO", "PISCO SUR"]
threshold_degradacion = 3
pozas = 10

cnxn_mio = pyodbc.connect('DRIVER={SQL Server};SERVER=srv-db-east-us003.database.windows.net;DATABASE=db_cfa_prd01;UID=userdbowner;PWD=$P4ssdbowner01#;')
cnxn_juv = pyodbc.connect('DRIVER={SQL Server};SERVER=srv-db-east-us-tasa-his-02.database.windows.net;DATABASE=db_bi_production_prd;UID=userpowerbi;PWD=#p4ssw0rdp0w3rb1#;')

#### Funciones calculadas

In [3]:
def weighted_average(df, values, weights):
    return (df[values] * df[weights]).sum() / df[weights].sum()

def estimate_tvn_con_frio(tdc):
    """
    Estima el TVN de las embarcaciones con RCW.
    """
    if tdc is None:
        return None
    if tdc <= 22:
        return 0.0008 * tdc**3 - 0.0332 * tdc**2 + 0.6275 * tdc + 14
    else:
        return 14.5 * np.exp(0.0136 * tdc) + 0.7


def estimate_tvn_sin_frio(tdc):
    """
    Estima el TVN de las embarcaciones sin RCW.
    """
    if tdc is None:
        return None
    if tdc <= 20:
        return 13.573 * np.exp(0.0386 * tdc) + 0.8
    else:
        return 14.573 * np.exp(0.043 * tdc) - 4.4

def process_discharge_plan_data(df):
    """
    Process the Discharge Plan data to extract relevant information.

    Parameters:
    df (DataFrame): The input DataFrame containing Discharge Plan data.

    Returns:
    DataFrame: A DataFrame with the processed Discharge Plan data.
    """
    df["ChataLinea"] = df["Chata"] + "-" + df["Linea"]
    df = df.sort_values(by=["Fecha_Inicio_Succion"]) 
    return df

def process_limpieza_tuberia_data(df, planta_selected):

    """
    Process the Limpieza Tuberia data to extract relevant information.

    Parameters:
    df (DataFrame): The input DataFrame containing Limpieza Tuberia data.

    Returns:
    DataFrame: A DataFrame with the processed Limpieza Tuberia data.
    """
    # Filter for specific plants
    to_rename = {
        "PLANTA": "Planta",
        "FlujoIC_operacion_m3":"ValorFlujoIngreso",
        "LimpiezaTuberia_m3":"LimpiezaTuberia",
        "FactorCorreccionAguaRecirculacion":"Factor de Corrección Agua Recirculación",
        "FCSolidosGrasayAguaBombeo":"FCSolidosGrasayAguaBombeo"
    }
    df = df.rename(columns=to_rename)
    df["Planta"] = df["Planta"].str.upper()
    mask = df["Planta"].isin([planta_selected])
    return df[mask].reset_index(drop=True)

def group_intervals(intervalos_df, stock_minimo_pama):

    resultados_pp_grouped = intervalos_df.groupby(['Intervalo Chata', "Intervalo Poza", "NumHora"]).apply(
        lambda x: pd.Series({
            'VolumenTotal': x['Volumen'].sum(),
            'VolumenTotalAB': x['VolumenTotalAB'].sum(),
            'TVNPonderado': weighted_average(x, 'TVNDescarga', 'Volumen'),
            'RAPDescarga': weighted_average(x, 'RAP', 'Volumen'),
            'RAPPonderado': weighted_average(x, 'RAP_PAMA', 'Volumen'),
        })
    ).reset_index()
    # resultados_pp_grouped["Intervalo Poza"] = intervalos_df[""]
    # resultados_pp_grouped = pd.merge(resultados_pp_grouped, intervalos_df[['Intervalo Chata', 'Intervalo Poza']], left_on='Intervalo Chata', right_on="Intervalo Chata", how='left')
    resultados_pp_grouped["VolumenAcumulado"] = resultados_pp_grouped["VolumenTotal"].cumsum()
    resultados_pp_grouped["VolumenPama"] = resultados_pp_grouped["VolumenTotalAB"].cumsum()
    # resultados_pp_grouped["VolumenPama"] = resultados_pp_grouped["RAPPonderado"] * resultados_pp_grouped["VolumenAcumulado"]
    resultados_pp_grouped["flg_min"] = 0
    mask = resultados_pp_grouped["VolumenPama"] > stock_minimo_pama
    resultados_pp_grouped.loc[mask, 'flg_min'] = 1

    return resultados_pp_grouped

def process_calidades(UmbralTVN_Calidades):
    """
    Process the Calidades data to extract relevant information.

    Parameters:
    df (DataFrame): The input DataFrame containing Calidades data.

    Returns:
    DataFrame: A DataFrame with the processed Calidades data.
    """
    # Filter for specific plants
    UmbralTVN_Calidades_copy = UmbralTVN_Calidades.copy()
    to_rename = {
        "PISCO_SUR": "PISCO SUR",
    }
    UmbralTVN_Calidades_copy = UmbralTVN_Calidades_copy.rename(columns=to_rename)
    return UmbralTVN_Calidades_copy

def process_chatas_lineas_data(df, planta_selected):
    """
    Process the Chatas Lineas data to extract relevant information.

    Parameters:
    df (DataFrame): The input DataFrame containing Chatas Lineas data.

    Returns:
    DataFrame: A DataFrame with the processed Chatas Lineas data.
    """
    # Filter for specific plants
    df["id_planta"] = df["id_planta"].str.upper()
    mask = df["id_planta"].isin([planta_selected])
    df["ChataLinea"] = df["name"] + "-" + df["id_linea"]
    return df[mask].reset_index(drop=True)

def process_condiciones_rap(df, planta_selected):
    """
    Process the Chatas Lineas data to extract relevant information.

    Parameters:
    df (DataFrame): The input DataFrame containing Chatas Lineas data.

    Returns:
    DataFrame: A DataFrame with the processed Chatas Lineas data.
    """
    df["Planta"] = df["Planta"].str.upper()    
    mask = df["Planta"].isin([planta_selected])

    df["UmbralTDC_Min"] = df["UmbralTDC_Min"].fillna(df["UmbralTDC_Max"])
    df["UmbralTDC_Max"] = df["UmbralTDC_Max"].fillna(df["UmbralTDC_Min"])
    return df[mask].reset_index(drop=True)

def process_velocidades(velocidades_volumen_df, planta):
    """
    Process the Velocidades data to extract relevant information.

    Parameters:
    df (DataFrame): The input DataFrame containing Velocidades data.

    Returns:
    DataFrame: A DataFrame with the processed Velocidades data.
    """
    # Filter for specific plants
    to_rename = {
        "GRASA_MP": "GRASA MP",
        "VOLUMEN_MIN_ARRANQUE":"VOLUMEN MIN ARRANQUE",
        "MIN_TM":"MIN - TM",
        "MAX_TM":"MAX - TM",
        "VELOCIDAD_ARRANQUE":"VELOCIDAD DE ARRANQUE",
        "VELOCIDAD_MAXIMA":"VELOCIDAD MAXIMA",
        "VELOCIDAD_CIERRE":"VELOCIDAD DE CIERRE",
    }
    velocidades_volumen_df = velocidades_volumen_df.rename(columns=to_rename)
    velocidades_volumen_df["PLANTA"] = velocidades_volumen_df["PLANTA"].str.upper()
    mask = velocidades_volumen_df["PLANTA"].isin([planta])
    return velocidades_volumen_df[mask].reset_index(drop=True)

def process_balance_ksa_data(balance_ksa, planta):
    """
    Process the Balance KSA data to extract relevant information.

    Parameters:
    df (DataFrame): The input DataFrame containing Balance KSA data.

    Returns:
    DataFrame: A DataFrame with the processed Balance KSA data.
    """
    balance_ksa_df = balance_ksa.copy()
    to_rename = {
        "PorcS_Flujo_licor":"%S Flujo de licor",
        "PorcS_Flujo_lodo":"%S Flujo de lodo",
        "PorcH_Flujo_KSA":"%H Flujo de KSA",
        "PorcS_Flujo_efluente":"%S Flujo de efluente",
    }
    balance_ksa_df = balance_ksa_df.rename(columns=to_rename)
    mask = balance_ksa_df["PLANTA"] == 'Pisco'
    balance_ksa_df.loc[mask, "PLANTA"] = 'PISCO SUR'
    # Filter for specific plants
    balance_ksa_df["PLANTA"] = balance_ksa_df["PLANTA"].str.upper()
    mask = balance_ksa_df["PLANTA"].isin([planta])
    return balance_ksa_df[mask].reset_index(drop=True)

def hallar_volumen_minimo_pama(velocidades_volumen_df, grasa_h75):
    mask = (np.vectorize(eval)(str(grasa_h75) + velocidades_volumen_df["GRASA MP"]))
    stock_agua_minimo = velocidades_volumen_df.loc[mask, "VOLUMEN MIN ARRANQUE"].values[0]
    return stock_agua_minimo

def evaluar_condicion(row, df_adicional):
    condicion_tdc_min = (np.vectorize(eval)((str(row["TDC_Descarga_Actual"]) + df_adicional["UmbralTDC_Min"])))
    condicion_tdc_max = (np.vectorize(eval)((str(row["TDC_Descarga_Actual"]) + df_adicional["UmbralTDC_Max"])))
    condicion_juveniles = (np.vectorize(eval)((str(row["JuvenilesPonderado"]) + df_adicional["RangoJuvenilesPonderado"])))
    mask = (df_adicional["EstadoDeFrio"] == row["TipoEP"]) & (np.vectorize(eval)((str(row["Volumen"]) + df_adicional["RangoToneladasDeclaradas"]))) & (df_adicional["SistemaAbsorbente"] == row["sistema_absorbente"]) & condicion_tdc_min & condicion_tdc_max & condicion_juveniles
    if len(df_adicional[mask]) > 0:
        return df_adicional.loc[mask, 'RAP'].values[0]
    else:
        return df_adicional.loc[df_adicional["EstadoDeFrio"] == row["TipoEP"], 'RAP'].mean()
        
def add_rap(pd_df, chatas_lineas_df, condiciones_rap, limpieza_tuberia_df):
    """
    Add RAP to the DataFrame.
    """
    # Assuming you have a DataFrame named df with the necessary columns
    # Calculate RAP based on the conditions provided
    pd_updated_df = pd.merge(pd_df, chatas_lineas_df[["ChataLinea", "sistema_absorbente"]], on="ChataLinea", how="left")
    
    pd_updated_df["RAP"] = pd_updated_df.apply(evaluar_condicion, axis=1, df_adicional=condiciones_rap)

    volumenes = pd_updated_df.groupby("Embarcacion", as_index=False).agg(
        VolTotal=("Volumen", "sum")
    )
    pd_updated_df = pd.merge(pd_updated_df, volumenes, on="Embarcacion", how='left')
    pd_updated_df["PctVol"] = pd_updated_df["Volumen"] / pd_updated_df["VolTotal"]
    pd_updated_df["LimpiezaTuberia"] = pd_updated_df["PctVol"] * limpieza_tuberia_df["LimpiezaTuberia"].values[0]
    pd_updated_df["FactorCorreccion"] = limpieza_tuberia_df["Factor de Corrección Agua Recirculación"].values[0]
    pd_updated_df["FCSolidosGrasayAguaBombeo"] = limpieza_tuberia_df["FCSolidosGrasayAguaBombeo"].values[0]
    return pd_updated_df

def generate_intervalo_horas(pd_df, fecha_inicio_name="Fecha_Inicio_Succion", fecha_fin_name="Fecha_Fin_Succion", nombre_intervalo_chata="Intervalo Chata", tolva_inicio_name="Inicio_Descarga_PD", tolva_fin_name="Fin_Descarga_PD", nombre_intervalo_poza="Intervalo Poza", hours_adicional=20):
    """
    Generate intervals of hours for the given DataFrame.

    Parameters:
    df (DataFrame): The input DataFrame containing Limpieza Tuberia data.

    Returns:
    DataFrame: A DataFrame with the generated intervals of hours.
    """
    horas_dia = pd.date_range(
        start=pd_df[fecha_inicio_name].min().floor('10min'),
        end=(pd_df[fecha_fin_name] + pd.Timedelta(hours=hours_adicional)).max().ceil("30min"),
        freq="H",
    )

    horas_dia_poza = pd.date_range(
        start=pd_df[tolva_inicio_name].min().floor('10min'),
        end=(pd_df[tolva_fin_name] + pd.Timedelta(hours=hours_adicional)).max().ceil("30min"),
        freq="H",
    )
    volumen_por_hora = pd.DataFrame(data=horas_dia, columns=[nombre_intervalo_chata])
    volumen_por_hora_poza = pd.DataFrame(data=horas_dia_poza, columns=[nombre_intervalo_poza])

    intervalos_df = pd.concat([volumen_por_hora, volumen_por_hora_poza], axis=1)
    intervalos_df["NumHora"] = np.arange(1, len(intervalos_df) + 1)
    return intervalos_df

def generar_perfil_velocidades(
    volumen: float,
    velocidades: Dict[str, float],
    duracion_paso_horas: float = 1.0
):
    """
    Genera un vector de velocidades (por hora) para cocinar todo el volumen,
    asegurando que el paso final (remanente) sea siempre menor que la velocidad de cierre.

    Parámetros:
    - volumen: Volumen total a cocinar (continuo).
    - velocidades: Diccionario con las cinco velocidades (ton/h):
        {
            'inicio': float,
            'subida': float,
            'crucero': float,
            'bajada': float,
            'cierre': float
        }
    - duracion_paso_horas: Duración de cada paso en horas (por defecto 1h).
    """
    # Convertimos a volumen procesado por paso
    v_ini = velocidades['Velocidad Inicio'] * duracion_paso_horas
    v_sub = velocidades['Velocidad Intermedia1'] * duracion_paso_horas
    v_cru = velocidades['Velocidad Maxima'] * duracion_paso_horas
    v_baj = velocidades['Velocidad Intermedia2'] * duracion_paso_horas
    v_cie = velocidades['Velocidad Final'] * duracion_paso_horas

    perfil: List[float] = []
    perfil_lbls: List[str]   = []
    rem = volumen

    # 1) Dos pasos de inicio (2h) si hay suficiente volumen
    if rem >= 2 * v_ini:
        perfil += [v_ini, v_ini]
        rem -= 2 * v_ini
        perfil_lbls += ['Velocidad Inicio', 'Velocidad Inicio']
    else:
        perfil.append(rem)
        perfil_lbls.append('Velocidad Final')
        return perfil, perfil_lbls

    # 2) Un paso de subida
    if rem >= v_sub:
        perfil.append(v_sub)
        perfil_lbls.append('Velocidad Intermedia1')
        rem -= v_sub
    else:
        perfil.append(rem)
        perfil_lbls.append('Velocidad Final')
        return perfil, perfil_lbls

    # 3) Reservamos bajada y cierre
    if rem < (v_baj + v_cie):
        # Si no alcanza para ambos completos, primero bajada parcial
        perfil.append(min(rem, v_baj))
        perfil_lbls.append('Velocidad Intermedia2')
        rem -= perfil[-1]
        if rem > 0:
            perfil.append(rem)  # remanente < v_cie necesariamente
            perfil_lbls.append('Velocidad Final')
        return perfil, perfil_lbls

    # Hay volumen suficiente para bajar y cerrar completos; los restamos para calcular crucero
    rem -= (v_baj + v_cie)

    # 4) Pasos completos de crucero
    n_cru = int(rem // v_cru)
    perfil += [v_cru] * n_cru
    perfil_lbls += ['Velocidad Maxima'] * n_cru
    rem -= n_cru * v_cru

    # 5) Pasos fijos: bajada y cierre
    perfil += [v_baj, v_cie]
    perfil_lbls += ['Velocidad Intermedia2', 'Velocidad Final']

    # 6) Ajuste de remanente para garantizar < v_cie
    if rem > 0:
        # Si rem ≥ v_cie, desglosamos en cierres completos + rem final
        if rem >= v_cie:
            n_cie_extra = int(rem // v_cie)
            perfil += [v_cie] * n_cie_extra
            perfil_lbls += ['Velocidad Final'] * n_cie_extra
            rem -= n_cie_extra * v_cie
        # Al final, rem < v_cie
        if rem > 0:
            perfil.append(rem)
            perfil_lbls.append('Velocidad Final')

    return perfil, perfil_lbls


def process_valores_fijos(ValoresFijosPP, planta):
    """
    Process the Valores Fijos data to extract relevant information.

    Parameters:
    df (DataFrame): The input DataFrame containing Valores Fijos data.

    Returns:
    DataFrame: A DataFrame with the processed Valores Fijos data.
    """
    to_rename = {
        "PROTEINA_HARINA": "PROTEINA-HARINA",
        "Ceniza_KK_PRENSA":"%Ceniza- KK PRENSA",
        "Grasa_KK_PRENSA":"%Grasa- KK PRENSA",
        "Humedad_KK_PRENSA":"%Humedad- KK PRENSA",
        "Proteina_KK_PRENSA":"% Proteina- KK PRENSA",
        "Grasa_KK_SEPARADORA":"%Grasa- KK SEPARADORA",
        "Ceniza_KK_SEPARADORA":"%Ceniza- KK SEPARADORA",
        "Proteina_KK_SEPARADORA":"% Proteina- KK SEPARADORA",
        "Humedad_KK_SEPARADORA":"%Humedad- KK SEPARADORA",
        "Grasa_CONCENTRADO":"%Grasa- CONCENTRADO",
        "Ceniza_CONCENTRADO":"%Ceniza- CONCENTRADO",
        "Proteina_CONCENTRADO":"% Proteina-CONCENTRADO",
        "Humedad_CONCENTRADO":"%Humedad- CONCENTRADO",
        "Ceniza_KSA":"%Ceniza- KSA",
        "Proteina_KSA":"% Proteina- KSA",
        "Humedad_KSA":"%Humedad- KSA",
    }
    ValoresFijosPP_copy = ValoresFijosPP.copy()
    ValoresFijosPP_copy = ValoresFijosPP_copy.rename(columns=to_rename)
    ValoresFijosPP_copy["PLANTA"] = ValoresFijosPP_copy["PLANTA"].str.upper()
    mask = ValoresFijosPP_copy["PLANTA"].isin([planta])
    return ValoresFijosPP_copy[mask].reset_index(drop=True)

def process_arranque(df, planta, fecha_batch):
    arranque_df = df.copy()
    to_rename = {
        "Planta":"PLANTA"
    }
    arranque_df = arranque_df.rename(to_rename, axis=1)
    arranque_df["PLANTA"] = arranque_df["PLANTA"].str.upper()
    mask = (arranque_df["PLANTA"].isin([planta])) & (arranque_df["FechaBatch"].astype(str) == fecha_batch)
    return arranque_df.loc[mask, "FechaAjustada"]

def hallar_velocidades_base(velocidades_volumen_df, grasa_h75, volumen_materia_prima):
    mask = (np.vectorize(eval)(str(grasa_h75) + velocidades_volumen_df["GRASA MP"])) & (velocidades_volumen_df["MIN - TM"] <= volumen_materia_prima) & (velocidades_volumen_df["MAX - TM"] >= volumen_materia_prima)
    v_inicial = velocidades_volumen_df.loc[mask, "VELOCIDAD DE ARRANQUE"].values[0]
    v_medio = velocidades_volumen_df.loc[mask, "VELOCIDAD MAXIMA"].values[0]
    v_intermedia_inicial =  velocidades_volumen_df.loc[mask, "VEL_INTERMEDIA1"].values[0]
    v_final = velocidades_volumen_df.loc[mask, "VELOCIDAD DE CIERRE"].values[0]
    v_intermedia_final = velocidades_volumen_df.loc[mask, "VEL_INTERMEDIA2"].values[0]
    return v_inicial, v_intermedia_inicial, v_medio, v_intermedia_final, v_final

def fill_intervals(intervalos_df, plan_descarga_df, cantidad_pozas=10, volumen_por_poza=250):

    intervalos_df_copy = intervalos_df.copy()
    for i in range(cantidad_pozas):
        intervalos_df_copy[f"Poza_{i+1}"] = 0
    plan_descarga_df_copy = plan_descarga_df.copy()
    for index, row in intervalos_df_copy.iterrows():
        start1 = row['Intervalo Chata']
        end1 = start1 + pd.Timedelta(hours=1)
        # print(index)
        start2 = plan_descarga_df_copy['Fecha_Inicio_Succion'].item()
        end2 = plan_descarga_df_copy['Fecha_Fin_Succion'].item()
        tdc = plan_descarga_df_copy['TDC_Descarga'].item()
        volumen_estimado = plan_descarga_df_copy['VolumenEstTM'].item()
        # velocidad = plan_descarga_df_copy['Velocidad'].item()
        velocidad = volumen_estimado / ((end2 - start2).total_seconds() / 3600)
        chatalinea = plan_descarga_df_copy['ChataLinea'].item()
        ep = plan_descarga_df_copy['Embarcacion'].item()
        tipoep = plan_descarga_df_copy['TIPO'].item()
        juveniles_ponderado = plan_descarga_df_copy['JuvenilesPonderado'].item()
        interseccion_inicio = max(start2, start1)
        interseccion_fin = min(end2, end1)
        
        # Calcular la duración de la intersección
        duracion_interseccion = max((interseccion_fin - interseccion_inicio).total_seconds(), 0)
        
        # Calcular la duración del rango original
        duracion_rango_original = (end1 - start1).total_seconds()
        
        # Calcular el porcentaje
        porcentaje = (duracion_interseccion / duracion_rango_original) if duracion_rango_original > 0 else 0

        intervalos_df_copy.at[index, 'Duracion'] = porcentaje
        intervalos_df_copy.at[index, 'Volumen'] = porcentaje * velocidad
        intervalos_df_copy.at[index, 'VelDesc'] = velocidad
        intervalos_df_copy.at[index, 'ChataLinea'] = chatalinea
        intervalos_df_copy.at[index, 'Embarcacion'] = ep
        intervalos_df_copy.at[index, 'TipoEP'] = tipoep
        intervalos_df_copy.at[index, 'TDC_Descarga_Inicial'] = tdc
        tvn_estimado = estimate_tvn_sin_frio(tdc) if tipoep == 'SF' else estimate_tvn_con_frio(tdc)
        intervalos_df_copy.at[index, 'TVN_Descarga_Inicial'] = tvn_estimado
        intervalos_df_copy.at[index, 'JuvenilesPonderado'] = juveniles_ponderado

        # intervalos_df_copy = almacenar_en_poza(intervalos_df_copy, tvn_estimado, porcentaje * velocidad)
    return intervalos_df_copy
# class ParametrosSistema:
#     """Parámetros constantes del sistema completo"""
    
#     # Materia Prima - Pescado
#     PESCADO_HUMEDAD = 75
#     PESCADO_CENIZAS = 4
    
#     # Materia Prima - Camaroncillo
#     CAMARONCILLO_HUMEDAD = 82.47
#     CAMARONCILLO_GRASA = 3.19
#     CAMARONCILLO_CENIZAS = 4.65
    
#     # Materia Prima - Escama
#     ESCAMA_HUMEDAD = 89
#     ESCAMA_GRASA = 0.19
#     ESCAMA_CENIZAS = 9
    
#     # Harina
#     HARINA_HUMEDAD = 7
#     HARINA_CENIZA = 0.07
#     HARINA_GRASA = 0.08
#     HARINA_PROTEINA = 0.65
    
#     # KK Prensa
#     KK_PRENSA_HUMEDAD = 0.42
#     KK_PRENSA_CENIZA = 0.115
#     KK_PRENSA_GRASA = 0.038
#     KK_PRENSA_PROTEINA = 1 - 0.42 - 0.115 - 0.038
    
#     # KK Separadora
#     KK_SEP_HUMEDAD = 0.62
#     KK_SEP_CENIZA = 0.04
#     KK_SEP_GRASA = 0.025
#     KK_SEP_PROTEINA = 1 - 0.62 - 0.04 - 0.025
    
#     # KK CC
#     KK_CC_HUMEDAD = 0.654
#     KK_CC_CENIZA = 0.066
#     KK_CC_GRASA = 0.035
#     KK_CC_PROTEINA = 1 - 0.654 - 0.066 - 0.035
    
#     # KSA
#     KSA_HUMEDAD = 0.75
#     KSA_CENIZA = 0.07
    
#     # Porcentajes de receta por calidad
#     PORCENTAJES_RECETA = {
#         'A': 0.06,
#         'B': 0.07,
#         'C': 0.10,
#         'D': 0.10
#     }
    
#     def get_porcentaje_receta(self, calidad: str) -> float:
#         return self.PORCENTAJES_RECETA.get(calidad, 0.06)

# class ParametrosSistema:
#     """Parámetros del sistema dependientes de la calidad de harina"""

#     # 🔹 Materia Prima - Pescado
#     PESCADO_HUMEDAD = {'A': 75, 'B': 74, 'C': 73, 'D': 72}
#     PESCADO_CENIZAS = {'A': 4.0, 'B': 4.2, 'C': 4.4, 'D': 4.6}

#     # 🔹 Materia Prima - Camaroncillo
#     CAMARONCILLO_HUMEDAD = {'A': 82.5, 'B': 83.0, 'C': 83.5, 'D': 84.0}
#     CAMARONCILLO_GRASA   = {'A': 3.2,  'B': 3.0,  'C': 2.8,  'D': 2.5}
#     CAMARONCILLO_CENIZAS = {'A': 4.6,  'B': 4.7,  'C': 4.8,  'D': 5.0}

#     # 🔹 Materia Prima - Escama
#     ESCAMA_HUMEDAD = {'A': 89.0, 'B': 88.5, 'C': 88.0, 'D': 87.5}
#     ESCAMA_GRASA   = {'A': 0.19, 'B': 0.20, 'C': 0.22, 'D': 0.25}
#     ESCAMA_CENIZAS = {'A': 9.0,  'B': 9.2,  'C': 9.5,  'D': 9.8}

#     # 🔹 Harina (fijos)
#     HARINA_HUMEDAD  = 7
#     HARINA_CENIZA   = 0.07
#     HARINA_GRASA    = 0.08
#     HARINA_PROTEINA = 0.65

#     # 🔹 KK Prensa
#     KK_PRENSA_HUMEDAD  = {'A': 0.42, 'B': 0.43, 'C': 0.44, 'D': 0.45}
#     KK_PRENSA_CENIZA   = {'A': 0.115,'B': 0.120,'C': 0.125,'D': 0.130}
#     KK_PRENSA_GRASA    = {'A': 0.038,'B': 0.037,'C': 0.036,'D': 0.035}
#     KK_PRENSA_PROTEINA = {'A': 0.427,'B': 0.413,'C': 0.399,'D': 0.385}

#     # 🔹 KK Separadora
#     KK_SEP_HUMEDAD  = {'A': 0.62, 'B': 0.63, 'C': 0.64, 'D': 0.65}
#     KK_SEP_CENIZA   = {'A': 0.04, 'B': 0.041,'C': 0.042,'D': 0.043}
#     KK_SEP_GRASA    = {'A': 0.025,'B': 0.024,'C': 0.023,'D': 0.022}
#     KK_SEP_PROTEINA = {'A': 0.315,'B': 0.305,'C': 0.295,'D': 0.285}

#     # 🔹 KK CC (Concentrado)
#     KK_CC_HUMEDAD  = {'A': 0.654, 'B': 0.66, 'C': 0.665, 'D': 0.67}
#     KK_CC_CENIZA   = {'A': 0.066, 'B': 0.067,'C': 0.068,'D': 0.069}
#     KK_CC_GRASA    = {'A': 0.035, 'B': 0.034,'C': 0.033,'D': 0.032}
#     KK_CC_PROTEINA = {'A': 0.245, 'B': 0.239,'C': 0.234,'D': 0.229}

#     # 🔹 KSA
#     KSA_HUMEDAD = {'A': 0.75, 'B': 0.76, 'C': 0.77, 'D': 0.78}
#     KSA_CENIZA  = {'A': 0.07, 'B': 0.071,'C': 0.072,'D': 0.073}

#     # 🔹 Porcentajes de receta
#     PORCENTAJES_RECETA = {'A': 0.06, 'B': 0.07, 'C': 0.10, 'D': 0.10}

#     # --- Métodos ---
#     def get(self, atributo: str, calidad: str):
#         """
#         Devuelve el valor del atributo según la calidad.
#         Si el atributo es un diccionario, busca la clave de calidad.
#         Si no existe la calidad, devuelve el primer valor del diccionario.
#         Si el atributo es un número fijo, devuelve el valor directamente.
#         """
#         valor = getattr(self, atributo)
#         if isinstance(valor, dict):
#             return valor.get(calidad, list(valor.values())[0])
#         return valor

#     def get_porcentaje_receta(self, calidad: str) -> float:
#         """Devuelve el porcentaje de receta según calidad."""
#         return self.PORCENTAJES_RECETA.get(calidad, 0.06)

class ParametrosSistema:
    """Parámetros del sistema: PESCADO dinámico, resto hardcodeado."""

    def __init__(self, df_valores, df_tvn_calidades=None):
        """
        Inicializa solo los valores de PESCADO desde el DataFrame ValoresFijosPP.
        El DataFrame debe tener columnas:
        ['PROTEINA-HARINA', 'Humedad_MP', 'Ceniza_MP']
        """
        self.df = df_valores
        self.df_tvn = df_tvn_calidades

        # --- Materia Prima - Pescado 
        self.PESCADO_HUMEDAD = self._extraer_diccionario('Humedad_MP')
        self.PESCADO_CENIZAS = self._extraer_diccionario('Ceniza_MP')

        # --- Materia Prima - Camaroncillo
        self.CAMARONCILLO_HUMEDAD = self._extraer_diccionario('Humedad_Camaroncillo')
        self.CAMARONCILLO_GRASA   = self._extraer_diccionario('Grasa_Camaroncillo')
        self.CAMARONCILLO_CENIZAS = self._extraer_diccionario('Ceniza_Camaroncillo')

        # --- Materia Prima - Escama 
        self.ESCAMA_HUMEDAD = self._extraer_diccionario('Humedad_Escamas')
        self.ESCAMA_GRASA   = self._extraer_diccionario('Grasa_Escamas')
        self.ESCAMA_CENIZAS = self._extraer_diccionario('Ceniza_Escamas')

        # --- Harina
        self.HARINA_HUMEDAD  = 7
        self.HARINA_CENIZA   = 0.07
        self.HARINA_GRASA    = 0.08
        self.HARINA_PROTEINA = 0.65

        # --- KK Prensa
        self.KK_PRENSA_HUMEDAD  = self._extraer_diccionario('%Humedad- KK PRENSA', dividir_por=100)
        self.KK_PRENSA_CENIZA   = self._extraer_diccionario('%Ceniza- KK PRENSA', dividir_por=100)
        self.KK_PRENSA_GRASA    = self._extraer_diccionario('%Grasa- KK PRENSA', dividir_por=100)
        self.KK_PRENSA_PROTEINA = self._extraer_diccionario('% Proteina- KK PRENSA', dividir_por=100)
        
        # --- KK Separadora
        self.KK_SEP_HUMEDAD  = self._extraer_diccionario('%Humedad- KK SEPARADORA', dividir_por=100)
        self.KK_SEP_CENIZA   = self._extraer_diccionario('%Ceniza- KK SEPARADORA', dividir_por=100)
        self.KK_SEP_GRASA    = self._extraer_diccionario('%Grasa- KK SEPARADORA', dividir_por=100)
        self.KK_SEP_PROTEINA = self._extraer_diccionario('% Proteina- KK SEPARADORA', dividir_por=100)

        # --- KK CC
        self.KK_CC_HUMEDAD  = self._extraer_diccionario('%Humedad- CONCENTRADO', dividir_por=100)
        self.KK_CC_CENIZA   = self._extraer_diccionario('%Ceniza- CONCENTRADO', dividir_por=100)
        self.KK_CC_GRASA    = self._extraer_diccionario('%Grasa- CONCENTRADO', dividir_por=100)
        self.KK_CC_PROTEINA = self._extraer_diccionario('% Proteina-CONCENTRADO', dividir_por=100)

        # --- KSA
        self.KSA_HUMEDAD = self._extraer_diccionario('%Humedad- KSA', dividir_por=100)
        self.KSA_CENIZA  = self._extraer_diccionario('%Ceniza- KSA', dividir_por=100)

        # --- Porcentajes de receta
        self.PORCENTAJES_RECETA = self._extraer_diccionario_tvn('CALIDAD', 'DOSIFICACION')

    # # ---------------------------------------------------------------------
    # def _extraer_diccionario(self, nombre_columna):
    #     """Extrae un diccionario {calidad: valor} desde el DataFrame."""
    #     if nombre_columna not in self.df.columns:
    #         return {}
    #     return self.df.set_index('PROTEINA-HARINA')[nombre_columna].dropna().to_dict()
    
    def _extraer_diccionario(self, nombre_columna, dividir_por=None):
        """
        Extrae un diccionario {calidad: valor} desde el DataFrame.
        Si 'dividir_por' se especifica, divide cada valor por ese número.
        """
        if nombre_columna not in self.df.columns:
            return {}
        dic = self.df.set_index('PROTEINA-HARINA')[nombre_columna].dropna().to_dict()
        if dividir_por:
            dic = {k: v / dividir_por for k, v in dic.items()}
        return dic

    def _extraer_diccionario_tvn(self, col_calidad, col_valor):
        """Extrae un diccionario {calidad: dosificación} desde UmbralTVN_Calidades."""
        if self.df_tvn is None:
            print("⚠️ No se proporcionó df_tvn_calidades.")
            return {}

        if col_calidad not in self.df_tvn.columns or col_valor not in self.df_tvn.columns:
            print(f"⚠️ Columnas '{col_calidad}' o '{col_valor}' no encontradas en df_tvn_calidades")
            return {}

        dic = dict(zip(
            self.df_tvn[col_calidad].astype(str).str.strip(),
            pd.to_numeric(self.df_tvn[col_valor], errors='coerce')
        ))

        # Limpieza: eliminar NaN o nulos
        dic = {k: v for k, v in dic.items() if pd.notna(v)}

        return dic
    
    # ---------------------------------------------------------------------
    def get(self, atributo: str, calidad: str):
        """
        Devuelve el valor del atributo según la calidad.
        Si el atributo es un diccionario, busca la clave de calidad.
        Si no existe, devuelve el primer valor.
        Si el atributo es un número, lo devuelve directo.
        """
        valor = getattr(self, atributo)
        if isinstance(valor, dict):
            return valor.get(calidad, list(valor.values())[0])
        return valor

    # ---------------------------------------------------------------------
    def get_porcentaje_receta(self, calidad: str) -> float:
        """Devuelve el porcentaje de receta según calidad."""
        return self.PORCENTAJES_RECETA.get(calidad, 0.06)

# class BalanceGeneralVectorizadoCompleto:
#     def __init__(self):
#         self.data = {}
#         self._longitud_principal = 0
#         self._parametros = ParametrosSistema()

class BalanceGeneralVectorizadoCompleto:
    def __init__(self, df_valores_fijos, df_tvn_calidades=None, speed_fixed=None):
        """
        Inicializa el balance con los parámetros del sistema cargados
        desde el DataFrame de valores fijos (ValoresFijosPP).
        """
        self.data = {}
        self._longitud_principal = 0
        self._parametros = ParametrosSistema(df_valores_fijos, df_tvn_calidades)
        self.speed_fixed = speed_fixed
        
    def _obtener_longitud_principal(self) -> int:
        """Obtiene la longitud del vector principal para referencia"""
        if 'intervalo_chata' in self.data and len(self.data['intervalo_chata']) > 0:
            return len(self.data['intervalo_chata'])
        return 0
    
    def _crear_vector_vacio(self, longitud: int, dtype=object) -> np.ndarray:
        """Crea un vector vacío del tamaño adecuado con NaN"""
        if dtype == object:
            return np.full(longitud, None, dtype=object)
        else:
            return np.full(longitud, np.nan, dtype=dtype)
    
    def _asegurar_longitud_vector(self, nombre_vector: str, longitud: int, dtype=object):
        """Asegura que un vector tenga la longitud correcta, creándolo si no existe"""
        if nombre_vector not in self.data:
            self.data[nombre_vector] = self._crear_vector_vacio(longitud, dtype)
        elif len(self.data[nombre_vector]) < longitud:
            vector_actual = self.data[nombre_vector]
            extension_dtype = vector_actual.dtype if vector_actual.dtype != object else dtype
            nan_extension = self._crear_vector_vacio(longitud - len(vector_actual), extension_dtype)
            self.data[nombre_vector] = np.concatenate([vector_actual, nan_extension])
        elif len(self.data[nombre_vector]) > longitud:
            self.data[nombre_vector] = self.data[nombre_vector][:longitud]
    
    def _asignar_datos_progresivos(self, columna: str, datos: list, inicio: int):
        """Asigna datos progresivamente a una columna manejando diferentes tipos"""
        if columna not in self.data:
            return
        
        longitud = len(self.data[columna])
        datos_a_asignar = min(len(datos), longitud - inicio)
        
        if datos_a_asignar > 0:
            try:
                datos_array = np.array(datos[:datos_a_asignar], dtype=self.data[columna].dtype)
                self.data[columna][inicio:inicio+datos_a_asignar] = datos_array
            except (ValueError, TypeError):
                for i in range(datos_a_asignar):
                    if inicio + i < longitud:
                        self.data[columna][inicio + i] = datos[i]
    
    def cargar_datos_completos_desde_excel(self, datos_excel: Dict[str, np.ndarray]):
        """Carga todos los datos """
        for key, value in datos_excel.items():
            self.data[key] = value
        
        self._longitud_principal = self._obtener_longitud_principal()
        
        for key in list(self.data.keys()):
            dtype_actual = self.data[key].dtype
            self._asegurar_longitud_vector(key, self._longitud_principal, dtype_actual)
    
    def inicializar_vectores_proceso(self):
        """Inicializa todos los vectores del proceso"""
        longitud = self._longitud_principal
        if longitud == 0:
            return
        
        # DEFINIR TODAS LAS COLUMNAS
        todas_las_columnas = [
            # Columnas básicas
            'intervalo_chata', 'intervalo_poza', 'descarga', 'avance_descarga',
            'grasa_mp', 'porcentaje_camaroncillo', 'velocidad',
            'velocidad_corregida_sa', 'velocidad_corregida_clarificador', 
            'velocidad_corregida_dosificacion',
            
            # Materia Prima - Pescado
            'pescado_tn2', 'pescado_hum_tn', 'pescado_porc_hum', 'pescado_gra_tn',
            'pescado_porc_gra', 'pescado_cen_tn', 'pescado_porc_cen',
            
            # Materia Prima - Camaroncillo
            'camaroncillo_tn', 'camaroncillo_hum_tn', 'camaroncillo_porc_hum',
            'camaroncillo_gra_tn', 'camaroncillo_porc_gra', 'camaroncillo_cen_tn',
            'camaroncillo_porc_cen',
            
            # Materia Prima - Escama
            'escama_tn', 'escama_hum_tn', 'escama_porc_hum', 'escama_gra_tn',
            'escama_porc_gra', 'escama_cen_tn', 'escama_porc_cen',
            
            # Materia Prima IC
            'mp_ic_tn', 'mp_ic_hum_tn', 'mp_ic_porc_hum', 'mp_ic_gra_tn',
            'mp_ic_porc_gra', 'mp_ic_cen_tn', 'mp_ic_porc_cen',
            
            # Acumulados y Stock
            'acum_alimentacion', 'acum_alimentacion_corregido_pama',
            'stock_pozas', 'stock_pozas_corregido_pama',
            
            # TVN y Sólidos
            'tvn_descarga', 'solidos_ab', 'grasa_ab', 'tvn_pozas',
            'tvn_pozas_corregido', 'tvn_ingreso_cocinas', 'tvn_ingreso_cocinas_corregido',
            'solidos_ab_ic', 'grasa_ab_ic', 'solidos_ab_ic_corregido_pama',
            'grasa_ab_corregido_pama',
            
            # RAP PAMA
            'rap_pama_desc', 'rap_pama',
            
            # Volúmenes generados
            'volumen_generado_ab_acumulado',
            
            # Calidad y Receta
            'calidad_harina', 'porcentaje_receta',
            
            # Flujos Teóricos
            'flujo_ksa_teorico', 'base_seca_teorica', 'flujo_ic_teorico',
            
            # Factores de Corrección
            'fc', 'flujo_ic_corregido', 'flujo_ksa_corregido', 'base_seca_corregida',
            'receta_corregida',
            
            # Límites de capacidad
            'limite_base_seca', 'limite_flujo_ic',
            
            # Correcciones por capacidad
            'fc2', 'base_seca_corregida_sa', 'flujo_ksa_corregido_sa',
            'flujo_ic_corregido_sa', 'receta_corregida_sa',
            
            # Correcciones por clarificador
            'fc3', 'flujo_ic_capacidad_clarificador', 
            'base_seca_capacidad_clarificador', 'flujo_ksa_capacidad_clarificador',
            
            # Correcciones por dosificación
            'receta_corregida_dosificacion', 'base_seca_corregida_dosificacion',
            'flujo_ksa_corregido_dosificacion', 'flujo_ic_corregido_dosificacion',
            
            # Clarificador
            'porcentaje_grasa_ingreso_clarificador', 'flujo_efluente',
            'porcentaje_grasa_efluente', 'porcentaje_solidos_efluente',
            'flujo_lodo', 'porcentaje_grasa_lodo', 'porcentaje_solidos_lodo',
            'flujo_licor', 'porcentaje_grasa_licor', 'porcentaje_solidos_licor',
            
            # Flujo KSA
            'flujo_ksa', 'porcentaje_grasa_ksa', 'porcentaje_solidos_ksa',
            'porcentaje_humedad_ksa',
            
            # TVN A
            # 'tvn_a',
            
            # KK Prensa
            'kk_prensa_tn', 'kk_prensa_porc_humedad', 'kk_prensa_hum_tn',
            'kk_prensa_porc_ceniza', 'kk_prensa_cen_tn', 'kk_prensa_porc_grasa',
            'kk_prensa_gra_tn', 'kk_prensa_porc_proteina', 'kk_prensa_pro_tn',
            
            # KK Separadora
            'kk_separadora_tn', 'kk_separadora_porc_humedad', 'kk_sep_hum_tn',
            'kk_separadora_porc_ceniza', 'kk_sep_cen_tn', 'kk_separadora_porc_grasa',
            'kk_sep_gra_tn', 'kk_separadora_porc_proteina', 'kk_sep_pro_tn',
            
            # KK CC
            'kk_cc_tn', 'kk_cc_porc_humedad_2', 'kk_cc_porc_humedad', 'kk_cc_hum_tn',
            'kk_cc_porc_ceniza_2', 'kk_cc_porc_ceniza', 'kk_cc_cen_tn',
            'kk_cc_porc_grasa', 'kk_cc_gra_tn', 'kk_cc_porc_proteina', 'kk_cc_pro_tn',
            
            # KSA
            'ksa_tn', 'ksa_porc_humedad', 'ksa_hum_tn', 'ksa_porc_ceniza',
            'ksa_cen_tn', 'ksa_porc_grasa', 'ksa_gra_tn', 'ksa_porc_proteina',
            'ksa_pro_tn',
            
            # Dosificación KSA
            'porcentaje_dosificacion_ksa2',
            
            # KK Integral
            'kk_integral_tn', 'kk_integral_hum_tn', 'kk_integral_cen_tn',
            'kk_integral_gra_tn', 'kk_integral_pro_tn', 'kk_integral_porc_humedad',
            'kk_integral_porc_ceniza', 'kk_integral_porc_grasa', 'kk_integral_porc_proteina',
            
            # Aportantes
            'aportante_porc_kp', 'aportante_porc_ks', 'aportante_porc_cc',
            
            # Harina
            'harina_porc_humedad', 'harina_porc_ceniza', 'harina_porc_grasa',
            'harina_porc_proteina', 'harina_porc_humedad_2', 'harina_porc_ceniza_2',
            'harina_porc_grasa_2', 'harina_porc_proteina_2',
            
            # Resultados finales
            'har_ton', 'rendimiento_harina', 'aceite_tn', 'rendimiento_aceite'
        ]
        
        # Inicializar todas las columnas
        for columna in todas_las_columnas:
            if columna not in self.data:
                # Determinar tipo basado en el nombre
                if any(x in columna.lower() for x in ['intervalo', 'calidad']):
                    dtype = object
                else:
                    dtype = float
                self._asegurar_longitud_vector(columna, longitud, dtype)
    
    def simular_progreso_completo(self):
        """Simula el progreso completo del proceso"""
        longitud = self._longitud_principal
        if longitud == 0:
            return
        
        inicio_proceso = 2  # Los datos empiezan alrededor de la fila 3
        
        if longitud <= inicio_proceso:
            return
        
        # COLUMNAS CONSTANTES
        if 'grasa_mp' in self.data:
            self.data['grasa_mp'][inicio_proceso:] = 3.0
        
        if 'porcentaje_camaroncillo' in self.data:
            self.data['porcentaje_camaroncillo'][inicio_proceso:] = 2.0
        
        # # VELOCIDAD - empieza vacía, luego se llena progresivamente
        # velocidades = [120.0, 120.0, 226.0, 226.0, 226.0, 210.0, 210.0, 210.0, 80.0, 80.0, 40.0, np.nan, np.nan]
        # # velocidades =
        # self._asignar_datos_progresivos('velocidad', velocidades, inicio_proceso)
        
        # VELOCIDAD - empieza vacía, luego se llena progresivamente
        velocidades = self.speed_fixed 

        # Asignar la lista al vector de velocidad
        self._asignar_datos_progresivos('velocidad', velocidades, inicio_proceso)
        
        # VELOCIDADES CORREGIDAS - siguen el mismo patrón que velocidad
        for columna in ['velocidad_corregida_sa', 'velocidad_corregida_clarificador', 
                       'velocidad_corregida_dosificacion']:
            if columna in self.data:
                datos_velocidad = [x if not pd.isna(x) else np.nan for x in self.data['velocidad'][inicio_proceso:inicio_proceso+len(velocidades)]]
                self._asignar_datos_progresivos(columna, datos_velocidad, inicio_proceso)
        
        # # TVN DESCARGA - datos progresivos
        # tvn_descarga = [21.88, 21.88, 22.23, 22.25, 22.03, 23.03, 22.69, 23.03, 22.69, 23.15]
        # self._asignar_datos_progresivos('tvn_descarga', tvn_descarga, inicio_proceso)
        
        # TVN DESCARGA - usar datos reales si existen
        if 'tvn_descarga' in self.data and not all(pd.isna(self.data['tvn_descarga'])):
            # Ya hay datos cargados desde Excel → mantenerlos
            print("✅ Usando TVN_DESCARGA desde datos_iniciales")
        else:
            # No hay datos, usar valores por defecto
            tvn_descarga = [21.88, 21.88, 22.23, 22.25, 22.03, 23.03, 22.69, 23.03, 22.69, 23.15]
            self._asignar_datos_progresivos('tvn_descarga', tvn_descarga, inicio_proceso)
            print("⚠️ Usando TVN_DESCARGA por defecto (no vino desde Excel)")

        
        
        # # TVN POZAS - datos progresivos
        # tvn_pozas = [21.88, 21.88, 22.23, 22.25, 22.03, 23.03, 22.69, 23.03, 22.69, 23.15]
        # self._asignar_datos_progresivos('tvn_pozas', tvn_pozas, inicio_proceso)
        
        if 'tvn_pozas' in self.data and not all(pd.isna(self.data['tvn_pozas'])):
            # Ya hay datos cargados desde Excel → mantenerlos
            print("✅ Usando TVN_POZAS desde la funcion cargar_datos_plan_descarga_y_tablas_mio")
        else:
            # No hay datos, usar valores por defecto
            tvn_descarga = [21.88, 21.88, 22.23, 22.25, 22.03, 23.03, 22.69, 23.03, 22.69, 23.15]
            self._asignar_datos_progresivos('tvn_descarga', tvn_descarga, inicio_proceso)
            print("⚠️ Usando TVN_DESCARGA por defecto (no vino desde Excel)")
        

        # TVN POZAS CORREGIDO - datos progresivos
        tvn_pozas_corregido = [21.88, 21.88, 22.23, 22.25, 22.03, 23.03, 22.69, 23.03, 22.69, 23.15]
        self._asignar_datos_progresivos('tvn_pozas_corregido', tvn_pozas_corregido, inicio_proceso)
        
        
        
        
        
        # # TVN INGRESO COCINAS - similar a TVN descarga
        # tvn_ingreso = [21.88, 22.33, 21.98, 22.15, 22.03, 22.15, 22.69, 23.15, 22.69, 23.15]
        # self._asignar_datos_progresivos('tvn_ingreso_cocinas', tvn_ingreso, inicio_proceso)
        # self._asignar_datos_progresivos('tvn_ingreso_cocinas_corregido', tvn_ingreso, inicio_proceso)
        
        print(self.data['tvn_ingreso_cocinas'])
        
        if 'tvn_ingreso_cocinas' in self.data and not all(pd.isna(self.data['tvn_ingreso_cocinas'])):
            # Ya hay datos cargados desde Excel → mantenerlos
            print("✅ Usando TVN_INGRESO_COCINAS desde la funcion cargar_datos_plan_descarga_y_tablas_mio")
        else:
            # No hay datos, usar valores por defecto
            tvn_ingreso = [21.88, 22.33, 21.98, 22.15, 22.03, 22.15, 22.69, 23.15, 22.69, 23.15]
            self._asignar_datos_progresivos('tvn_ingreso_cocinas', tvn_ingreso, inicio_proceso)
            print("⚠️ Usando TVN_INGRESO por defecto (no vino desde Excel)")
        
        
        
        # CALIDAD HARINA - valores categóricos (strings)
        calidades = ['A', 'A', 'A', 'A', 'A', 'C', 'C', 'C', 'D', 'D', 'D', 'D']
        self._asignar_datos_progresivos('calidad_harina', calidades, inicio_proceso)
        
        
        print(self.data['calidades'])
        
        if 'calidades' in self.data and not all(pd.isna(self.data['calidades'])):
            # Ya hay datos cargados desde Excel → mantenerlos
            print("✅ Usando CALIDAD desde la funcion cargar_datos_plan_descarga_y_tablas_mio")
        else:
            # No hay datos, usar valores por defecto
            calidades = ['A', 'A', 'A', 'A', 'A', 'C', 'C', 'C', 'D', 'D', 'D', 'D']
            self._asignar_datos_progresivos('calidad_harina', calidades, inicio_proceso)
            print("⚠️ Usando TVN_INGRESO por defecto (no vino desde Excel)")
        
        
        
        # PORCENTAJE RECETA - basado en calidad harina
        if 'calidad_harina' in self.data and 'porcentaje_receta' in self.data:
            for i in range(inicio_proceso, min(longitud, inicio_proceso + len(calidades))):
                if (i < len(self.data['calidad_harina']) and 
                    self.data['calidad_harina'][i] is not None and 
                    not pd.isna(self.data['calidad_harina'][i])):
                    
                    calidad = self.data['calidad_harina'][i]
                    porcentaje = self._parametros.get_porcentaje_receta(calidad)
                    if i < len(self.data['porcentaje_receta']):
                        self.data['porcentaje_receta'][i] = porcentaje
        
        # ACUMULADOS - se van acumulando
        if 'descarga' in self.data and 'acum_alimentacion' in self.data:
            descargas_validas = []
            indices_validos = []
            
            for i in range(inicio_proceso, min(longitud, len(self.data['velocidad_corregida_dosificacion']))):
                if (not pd.isna(self.data['velocidad_corregida_dosificacion'][i]) and 
                    self.data['velocidad_corregida_dosificacion'][i] is not None):
                    descargas_validas.append(float(self.data['velocidad_corregida_dosificacion'][i]))
                    indices_validos.append(i)
            
            if descargas_validas:
                acumulados = np.cumsum(descargas_validas)
                for idx, acum in zip(indices_validos, acumulados):
                    if idx < len(self.data['acum_alimentacion']):
                        self.data['acum_alimentacion'][idx] = acum
                        self.data['acum_alimentacion_corregido_pama'][idx] = acum # ´TODO: Cambiar a valor corregido si es necesario
                        self.data['stock_pozas'][idx] = self.data['avance_descarga'][idx] - acum
                        self.data['stock_pozas_corregido_pama'][idx] = self.data['avance_descarga'][idx] - acum
            
        
        # CONSTANTES DEL SISTEMA
        constantes = {
            'rap_pama_desc': 1.84,
            'rap_pama': 1.84,
            'limite_base_seca': 4.5,
            'limite_flujo_ic': 550.0,
            'fc': 1.0,
            'fc2': 1.0,
            'fc3': 550.0/597.62,
            'aportante_porc_kp': 23.0,
            'aportante_porc_ks': 8.0,
            'aportante_porc_cc': 14.0,
            'harina_porc_humedad': 7.0,
            'harina_porc_ceniza': 0.07,
            'harina_porc_grasa': 0.08,
            'harina_porc_proteina': 0.65
        }
        
        for columna, valor in constantes.items():
            if columna in self.data:
                # Asignar a todas las filas desde inicio_proceso
                for i in range(inicio_proceso, longitud):
                    if i < len(self.data[columna]):
                        self.data[columna][i] = valor

    # CALCULAR VECTORES DE VOLUMEN GENERADO AB ACUMULADO TODO:
        for i in range(inicio_proceso, longitud):
            # if i < len(self.data[columna]):
                self.data["volumen_generado_ab_acumulado"][i] = self.data["rap_pama"][i] * self.data["avance_descarga"][i]
        
    # def calcular_vectores_materia_prima(self):
    #     """Calcula todos los vectores de materia prima de manera segura"""
    #     longitud = self._longitud_principal
    #     if longitud == 0:
    #         return
        
    #     # Usar enfoque iterativo para evitar problemas de máscaras
    #     for i in range(longitud):
    #         if (i < len(self.data['velocidad_corregida_dosificacion']) and 
    #             not pd.isna(self.data['velocidad_corregida_dosificacion'][i]) and 
    #             self.data['velocidad_corregida_dosificacion'][i] > 0 and
    #             i < len(self.data['grasa_mp']) and
    #             not pd.isna(self.data['grasa_mp'][i]) and
    #             i < len(self.data['porcentaje_camaroncillo']) and
    #             not pd.isna(self.data['porcentaje_camaroncillo'][i])):
                
    #             try:
    #                 velocidad = float(self.data['velocidad_corregida_dosificacion'][i])
    #                 grasa_mp = float(self.data['grasa_mp'][i])
    #                 porc_camaroncillo = float(self.data['porcentaje_camaroncillo'][i])
                    
    #                 # PESCADO
    #                 if i < len(self.data['pescado_tn2']):
    #                     self.data['pescado_tn2'][i] = velocidad
                    
    #                 if i < len(self.data['pescado_hum_tn']):
    #                     self.data['pescado_hum_tn'][i] = velocidad * self._parametros.PESCADO_HUMEDAD / 100
                    
    #                 if i < len(self.data['pescado_gra_tn']):
    #                     self.data['pescado_gra_tn'][i] = velocidad * grasa_mp / 100
                    
    #                 if i < len(self.data['pescado_cen_tn']):
    #                     self.data['pescado_cen_tn'][i] = velocidad * self._parametros.PESCADO_CENIZAS / 100
                    
    #                 # Porcentajes Pescado
    #                 if i < len(self.data['pescado_porc_hum']):
    #                     self.data['pescado_porc_hum'][i] = self._parametros.PESCADO_HUMEDAD
                    
    #                 if i < len(self.data['pescado_porc_gra']):
    #                     self.data['pescado_porc_gra'][i] = grasa_mp
                    
    #                 if i < len(self.data['pescado_porc_cen']):
    #                     self.data['pescado_porc_cen'][i] = self._parametros.PESCADO_CENIZAS
                    
    #                 # CAMARONCILLO
    #                 camaroncillo_tn = velocidad * porc_camaroncillo / 100
                    
    #                 if i < len(self.data['camaroncillo_tn']):
    #                     self.data['camaroncillo_tn'][i] = camaroncillo_tn
                    
    #                 if i < len(self.data['camaroncillo_hum_tn']):
    #                     self.data['camaroncillo_hum_tn'][i] = camaroncillo_tn * self._parametros.CAMARONCILLO_HUMEDAD / 100
                    
    #                 if i < len(self.data['camaroncillo_gra_tn']):
    #                     self.data['camaroncillo_gra_tn'][i] = camaroncillo_tn * self._parametros.CAMARONCILLO_GRASA / 100
                    
    #                 if i < len(self.data['camaroncillo_cen_tn']):
    #                     self.data['camaroncillo_cen_tn'][i] = camaroncillo_tn * self._parametros.CAMARONCILLO_CENIZAS / 100
                    
    #                 # Porcentajes Camaroncillo
    #                 if i < len(self.data['camaroncillo_porc_hum']):
    #                     self.data['camaroncillo_porc_hum'][i] = self._parametros.CAMARONCILLO_HUMEDAD
                    
    #                 if i < len(self.data['camaroncillo_porc_gra']):
    #                     self.data['camaroncillo_porc_gra'][i] = self._parametros.CAMARONCILLO_GRASA
                    
    #                 if i < len(self.data['camaroncillo_porc_cen']):
    #                     self.data['camaroncillo_porc_cen'][i] = self._parametros.CAMARONCILLO_CENIZAS
                    
    #                 # ESCAMA
    #                 escama_tn = 0.02 * (camaroncillo_tn + velocidad)
                    
    #                 if i < len(self.data['escama_tn']):
    #                     self.data['escama_tn'][i] = escama_tn
                    
    #                 if i < len(self.data['escama_hum_tn']):
    #                     self.data['escama_hum_tn'][i] = escama_tn * self._parametros.ESCAMA_HUMEDAD / 100
                    
    #                 if i < len(self.data['escama_gra_tn']):
    #                     self.data['escama_gra_tn'][i] = escama_tn * self._parametros.ESCAMA_GRASA / 100
                    
    #                 if i < len(self.data['escama_cen_tn']):
    #                     self.data['escama_cen_tn'][i] = escama_tn * self._parametros.ESCAMA_CENIZAS / 100
                    
    #                 # Porcentajes Escama
    #                 if i < len(self.data['escama_porc_hum']):
    #                     self.data['escama_porc_hum'][i] = self._parametros.ESCAMA_HUMEDAD
                    
    #                 if i < len(self.data['escama_porc_gra']):
    #                     self.data['escama_porc_gra'][i] = self._parametros.ESCAMA_GRASA
                    
    #                 if i < len(self.data['escama_porc_cen']):
    #                     self.data['escama_porc_cen'][i] = self._parametros.ESCAMA_CENIZAS
                    
    #                 # MATERIA PRIMA INGRESO COCINAS
    #                 mp_ic_tn = velocidad + camaroncillo_tn + escama_tn
                    
    #                 if i < len(self.data['mp_ic_tn']):
    #                     self.data['mp_ic_tn'][i] = mp_ic_tn
                    
    #                 if i < len(self.data['mp_ic_hum_tn']):
    #                     self.data['mp_ic_hum_tn'][i] = (self.data['pescado_hum_tn'][i] + 
    #                                                   self.data['camaroncillo_hum_tn'][i] + 
    #                                                   self.data['escama_hum_tn'][i])
                    
    #                 if i < len(self.data['mp_ic_gra_tn']):
    #                     self.data['mp_ic_gra_tn'][i] = (self.data['pescado_gra_tn'][i] + 
    #                                                   self.data['camaroncillo_gra_tn'][i] + 
    #                                                   self.data['escama_gra_tn'][i])
                    
    #                 if i < len(self.data['mp_ic_cen_tn']):
    #                     self.data['mp_ic_cen_tn'][i] = (self.data['pescado_cen_tn'][i] + 
    #                                                   self.data['camaroncillo_cen_tn'][i] + 
    #                                                   self.data['escama_cen_tn'][i])
                    
    #                 # PORCENTAJES MP IC
    #                 if mp_ic_tn > 0:
    #                     if i < len(self.data['mp_ic_porc_hum']):
    #                         self.data['mp_ic_porc_hum'][i] = (self.data['mp_ic_hum_tn'][i] / mp_ic_tn) * 100
                        
    #                     if i < len(self.data['mp_ic_porc_gra']):
    #                         self.data['mp_ic_porc_gra'][i] = (self.data['mp_ic_gra_tn'][i] / mp_ic_tn) * 100
                        
    #                     if i < len(self.data['mp_ic_porc_cen']):
    #                         self.data['mp_ic_porc_cen'][i] = (self.data['mp_ic_cen_tn'][i] / mp_ic_tn) * 100
                            
    #             except (TypeError, ValueError, ZeroDivisionError) as e:
    #                 # Silenciosamente continuar con el siguiente índice
    #                 print("ERROR en cálculo de materia prima en índice {}: {}".format(i, e))
    #                 continue

    def calcular_vectores_materia_prima(self):
        """Calcula todos los vectores de materia prima usando los parámetros según calidad."""
        longitud = self._longitud_principal
        if longitud == 0:
            return

        for i in range(longitud):

            # Validar si hay datos válidos para procesar esta fila
            if (
                i < len(self.data['velocidad_corregida_dosificacion'])
                and not pd.isna(self.data['velocidad_corregida_dosificacion'][i])
                and self.data['velocidad_corregida_dosificacion'][i] > 0
                and i < len(self.data['grasa_mp'])
                and not pd.isna(self.data['grasa_mp'][i])
                and i < len(self.data['porcentaje_camaroncillo'])
                and not pd.isna(self.data['porcentaje_camaroncillo'][i])
            ):

                try:
                    velocidad = float(self.data['velocidad_corregida_dosificacion'][i])
                    grasa_mp = float(self.data['grasa_mp'][i])
                    porc_camaroncillo = float(self.data['porcentaje_camaroncillo'][i])

                    # Obtener la calidad actual (por ejemplo A, B, C, D)
                    calidad = None
                    if 'calidad_harina' in self.data and i < len(self.data['calidad_harina']):
                        calidad = self.data['calidad_harina'][i]

                    # Si no hay calidad definida, usar un valor por defecto (por ejemplo A)
                    if calidad is None or pd.isna(calidad):
                        calidad = 'A'

                    # --- 🔹 PESCADO ---
                    hum_pescado = self._parametros.get('PESCADO_HUMEDAD', calidad)
                    cen_pescado = self._parametros.get('PESCADO_CENIZAS', calidad)

                    if i < len(self.data['pescado_tn2']):
                        self.data['pescado_tn2'][i] = velocidad

                    if i < len(self.data['pescado_hum_tn']):
                        self.data['pescado_hum_tn'][i] = velocidad * hum_pescado / 100

                    if i < len(self.data['pescado_gra_tn']):
                        self.data['pescado_gra_tn'][i] = velocidad * grasa_mp / 100

                    if i < len(self.data['pescado_cen_tn']):
                        self.data['pescado_cen_tn'][i] = velocidad * cen_pescado / 100

                    # Porcentajes Pescado
                    if i < len(self.data['pescado_porc_hum']):
                        self.data['pescado_porc_hum'][i] = hum_pescado
                    if i < len(self.data['pescado_porc_gra']):
                        self.data['pescado_porc_gra'][i] = grasa_mp
                    if i < len(self.data['pescado_porc_cen']):
                        self.data['pescado_porc_cen'][i] = cen_pescado

                    # --- 🔹 CAMARONCILLO ---
                    hum_cama = self._parametros.get('CAMARONCILLO_HUMEDAD', calidad)
                    gra_cama = self._parametros.get('CAMARONCILLO_GRASA', calidad)
                    cen_cama = self._parametros.get('CAMARONCILLO_CENIZAS', calidad)

                    camaroncillo_tn = velocidad * porc_camaroncillo / 100

                    if i < len(self.data['camaroncillo_tn']):
                        self.data['camaroncillo_tn'][i] = camaroncillo_tn
                    if i < len(self.data['camaroncillo_hum_tn']):
                        self.data['camaroncillo_hum_tn'][i] = camaroncillo_tn * hum_cama / 100
                    if i < len(self.data['camaroncillo_gra_tn']):
                        self.data['camaroncillo_gra_tn'][i] = camaroncillo_tn * gra_cama / 100
                    if i < len(self.data['camaroncillo_cen_tn']):
                        self.data['camaroncillo_cen_tn'][i] = camaroncillo_tn * cen_cama / 100

                    if i < len(self.data['camaroncillo_porc_hum']):
                        self.data['camaroncillo_porc_hum'][i] = hum_cama
                    if i < len(self.data['camaroncillo_porc_gra']):
                        self.data['camaroncillo_porc_gra'][i] = gra_cama
                    if i < len(self.data['camaroncillo_porc_cen']):
                        self.data['camaroncillo_porc_cen'][i] = cen_cama

                    # --- 🔹 ESCAMA ---
                    hum_esc = self._parametros.get('ESCAMA_HUMEDAD', calidad)
                    gra_esc = self._parametros.get('ESCAMA_GRASA', calidad)
                    cen_esc = self._parametros.get('ESCAMA_CENIZAS', calidad)

                    escama_tn = 0.02 * (camaroncillo_tn + velocidad)

                    if i < len(self.data['escama_tn']):
                        self.data['escama_tn'][i] = escama_tn
                    if i < len(self.data['escama_hum_tn']):
                        self.data['escama_hum_tn'][i] = escama_tn * hum_esc / 100
                    if i < len(self.data['escama_gra_tn']):
                        self.data['escama_gra_tn'][i] = escama_tn * gra_esc / 100
                    if i < len(self.data['escama_cen_tn']):
                        self.data['escama_cen_tn'][i] = escama_tn * cen_esc / 100

                    if i < len(self.data['escama_porc_hum']):
                        self.data['escama_porc_hum'][i] = hum_esc
                    if i < len(self.data['escama_porc_gra']):
                        self.data['escama_porc_gra'][i] = gra_esc
                    if i < len(self.data['escama_porc_cen']):
                        self.data['escama_porc_cen'][i] = cen_esc

                    # --- 🔹 MATERIA PRIMA INGRESO COCINAS ---
                    mp_ic_tn = velocidad + camaroncillo_tn + escama_tn
                    if i < len(self.data['mp_ic_tn']):
                        self.data['mp_ic_tn'][i] = mp_ic_tn

                    if i < len(self.data['mp_ic_hum_tn']):
                        self.data['mp_ic_hum_tn'][i] = (
                            self.data['pescado_hum_tn'][i]
                            + self.data['camaroncillo_hum_tn'][i]
                            + self.data['escama_hum_tn'][i]
                        )

                    if i < len(self.data['mp_ic_gra_tn']):
                        self.data['mp_ic_gra_tn'][i] = (
                            self.data['pescado_gra_tn'][i]
                            + self.data['camaroncillo_gra_tn'][i]
                            + self.data['escama_gra_tn'][i]
                        )

                    if i < len(self.data['mp_ic_cen_tn']):
                        self.data['mp_ic_cen_tn'][i] = (
                            self.data['pescado_cen_tn'][i]
                            + self.data['camaroncillo_cen_tn'][i]
                            + self.data['escama_cen_tn'][i]
                        )

                    if mp_ic_tn > 0:
                        if i < len(self.data['mp_ic_porc_hum']):
                            self.data['mp_ic_porc_hum'][i] = (self.data['mp_ic_hum_tn'][i] / mp_ic_tn) * 100
                        if i < len(self.data['mp_ic_porc_gra']):
                            self.data['mp_ic_porc_gra'][i] = (self.data['mp_ic_gra_tn'][i] / mp_ic_tn) * 100
                        if i < len(self.data['mp_ic_porc_cen']):
                            self.data['mp_ic_porc_cen'][i] = (self.data['mp_ic_cen_tn'][i] / mp_ic_tn) * 100

                except (TypeError, ValueError, ZeroDivisionError) as e:
                    print(f"⚠️ ERROR en índice {i}: {e}")
                    continue

    def calcular_vectores_proceso(self):
        """Calcula vectores del proceso principal de manera segura"""
        longitud = self._longitud_principal
        if longitud == 0:
            return
        
        for i in range(longitud):
            try:
                # SÓLIDOS AB
                if (i < len(self.data['grasa_mp']) and 
                    i < len(self.data['tvn_descarga']) and
                    not pd.isna(self.data['grasa_mp'][i]) and
                    not pd.isna(self.data['tvn_descarga'][i])):
                    
                    tvn = float(self.data['tvn_descarga'][i])
                    grasa = float(self.data['grasa_mp'][i])
                    
                    solidos_ab = (-445.24 + tvn * 294 - 606.62 * grasa + 88.804 * grasa**2)
                    grasa_ab = -47.169 + 0.943 * solidos_ab
                    
                    if i < len(self.data['solidos_ab']):
                        self.data['solidos_ab'][i] = solidos_ab
                    if i < len(self.data['grasa_ab']):
                        self.data['grasa_ab'][i] = grasa_ab
                
                # SÓLIDOS AB IC
                if (i < len(self.data['grasa_mp']) and 
                    i < len(self.data['tvn_ingreso_cocinas']) and
                    not pd.isna(self.data['grasa_mp'][i]) and
                    not pd.isna(self.data['tvn_ingreso_cocinas'][i])):
                    
                    tvn_ic = float(self.data['tvn_ingreso_cocinas'][i])
                    grasa = float(self.data['grasa_mp'][i])
                    
                    solidos_ab_ic = (-445.24 + tvn_ic * 294 - 606.62 * grasa + 88.804 * grasa**2)
                    grasa_ab_ic = -47.169 + 0.943 * solidos_ab_ic
                    
                    if i < len(self.data['solidos_ab_ic']):
                        self.data['solidos_ab_ic'][i] = solidos_ab_ic
                    if i < len(self.data['grasa_ab_ic']):
                        self.data['grasa_ab_ic'][i] = grasa_ab_ic
                
                # SÓLIDOS AB IC CORREGIDO
                if (i < len(self.data['grasa_mp']) and 
                    i < len(self.data['tvn_ingreso_cocinas_corregido']) and
                    not pd.isna(self.data['grasa_mp'][i]) and
                    not pd.isna(self.data['tvn_ingreso_cocinas_corregido'][i])):
                    
                    tvn_ic_corr = float(self.data['tvn_ingreso_cocinas_corregido'][i])
                    grasa = float(self.data['grasa_mp'][i])
                    
                    solidos_ab_ic_corr = (-445.24 + tvn_ic_corr * 294 - 606.62 * grasa + 88.804 * grasa**2)
                    grasa_ab_corr = -47.169 + 0.943 * solidos_ab_ic_corr
                    
                    if i < len(self.data['solidos_ab_ic_corregido_pama']):
                        self.data['solidos_ab_ic_corregido_pama'][i] = solidos_ab_ic_corr
                    if i < len(self.data['grasa_ab_corregido_pama']):
                        self.data['grasa_ab_corregido_pama'][i] = grasa_ab_corr
                        
            except (TypeError, ValueError) as e:
                continue

    def calcular_vectores_flujos(self):
        """Calcula vectores de flujos de manera segura"""
        longitud = self._longitud_principal
        if longitud == 0:
            return
        
        for i in range(longitud):
            try:
                if (i < len(self.data['velocidad']) and 
                    i < len(self.data['porcentaje_receta']) and
                    i < len(self.data['solidos_ab_ic']) and
                    not pd.isna(self.data['velocidad'][i]) and
                    self.data['velocidad'][i] > 0 and
                    not pd.isna(self.data['porcentaje_receta'][i]) and
                    not pd.isna(self.data['solidos_ab_ic'][i])):
                    
                    velocidad = float(self.data['velocidad'][i])
                    porcentaje_receta = float(self.data['porcentaje_receta'][i])
                    solidos_ab_ic = float(self.data['solidos_ab_ic'][i])
                    fc = float(self.data['fc'][i]) if (i < len(self.data['fc']) and not pd.isna(self.data['fc'][i])) else 1.0
                    
                    # FLUJOS TEÓRICOS
                    flujo_ksa_teorico = porcentaje_receta * velocidad
                    base_seca_teorica = flujo_ksa_teorico / 4
                    
                    if i < len(self.data['flujo_ksa_teorico']):
                        self.data['flujo_ksa_teorico'][i] = flujo_ksa_teorico
                    if i < len(self.data['base_seca_teorica']):
                        self.data['base_seca_teorica'][i] = base_seca_teorica
                    
                    # FLUJO IC TEÓRICO
                    if solidos_ab_ic > 0:
                        flujo_ic_teorico = (base_seca_teorica * 1000000) / solidos_ab_ic
                        if i < len(self.data['flujo_ic_teorico']):
                            self.data['flujo_ic_teorico'][i] = flujo_ic_teorico
                        
                        # FLUJOS CORREGIDOS
                        flujo_ic_corregido = flujo_ic_teorico * fc
                        if i < len(self.data['flujo_ic_corregido']):
                            self.data['flujo_ic_corregido'][i] = flujo_ic_corregido
                        
                        # FLUJO KSA CORREGIDO
                        flujo_ksa_corregido = (flujo_ic_corregido * solidos_ab_ic * 4) / 1000000
                        base_seca_corregida = flujo_ksa_corregido / 4
                        receta_corregida = flujo_ksa_corregido / velocidad
                        
                        # FLUJOS CORREGIDOS PAMA                        
                        if i < len(self.data['flujo_ksa_corregido']):
                            self.data['flujo_ksa_corregido'][i] = flujo_ksa_corregido
                        if i < len(self.data['base_seca_corregida']):
                            self.data['base_seca_corregida'][i] = base_seca_corregida
                        if i < len(self.data['receta_corregida']):
                            self.data['receta_corregida'][i] = receta_corregida

                        # FLUJOS CORREGIDOS SA
                        if i < len(self.data['base_seca_corregida_sa']):
                            self.data['base_seca_corregida_sa'][i] = base_seca_corregida
                        if i < len(self.data['flujo_ksa_corregido_sa']):
                            self.data['flujo_ksa_corregido_sa'][i] = flujo_ksa_corregido 
                        if i < len(self.data['flujo_ic_corregido_sa']):
                            self.data['flujo_ic_corregido_sa'][i] = flujo_ic_corregido
                        if i < len(self.data['receta_corregida_sa']):
                            self.data['receta_corregida_sa'][i] = receta_corregida

                        # FLUJOS CORREGIDOS CLARIFICADOR
                        if i < len(self.data['flujo_ic_capacidad_clarificador']):
                            self.data['flujo_ic_capacidad_clarificador'][i] = flujo_ic_corregido 
                        if i < len(self.data['base_seca_capacidad_clarificador']):
                            self.data['base_seca_capacidad_clarificador'][i] = base_seca_corregida
                        if i < len(self.data['flujo_ksa_capacidad_clarificador']):
                            self.data['flujo_ksa_capacidad_clarificador'][i] = flujo_ksa_corregido   

                        # FLUJOS CORREGIDOS DOSIFICACIÓN
                        if i < len(self.data['receta_corregida_dosificacion']):
                            self.data['receta_corregida_dosificacion'][i] = receta_corregida
                        if i < len(self.data['base_seca_corregida_dosificacion']):
                            self.data['base_seca_corregida_dosificacion'][i] = base_seca_corregida
                        if i < len(self.data['flujo_ksa_corregido_dosificacion']):
                            self.data['flujo_ksa_corregido_dosificacion'][i] = flujo_ksa_corregido
                        if i < len(self.data['flujo_ic_corregido_dosificacion']):
                            self.data['flujo_ic_corregido_dosificacion'][i] = flujo_ic_corregido                         
                                                                                                                                                     
                            
            except (TypeError, ValueError, ZeroDivisionError) as e:
                continue

    # def calcular_vectores_productos(self):
    #     """Calcula vectores de productos de manera segura"""
    #     longitud = self._longitud_principal
    #     if longitud == 0:
    #         return
        
    #     for i in range(longitud):
    #         try:
    #             if (i < len(self.data['mp_ic_tn']) and 
    #                 not pd.isna(self.data['mp_ic_tn'][i]) and 
    #                 self.data['mp_ic_tn'][i] > 0 and
    #                 i < len(self.data['aportante_porc_kp']) and
    #                 i < len(self.data['aportante_porc_ks']) and
    #                 i < len(self.data['aportante_porc_cc'])):
                    
    #                 mp_ic_tn = float(self.data['mp_ic_tn'][i])
    #                 aportante_kp = float(self.data['aportante_porc_kp'][i])
    #                 aportante_ks = float(self.data['aportante_porc_ks'][i])
    #                 aportante_cc = float(self.data['aportante_porc_cc'][i])
                    
    #                 # VECTORES PREVIOS TODO:
    #                 if i < len(self.data['porcentaje_grasa_ingreso_clarificador']):
    #                     self.data['porcentaje_grasa_ingreso_clarificador'][i] = 0.5 * self.data['grasa_ab_corregido_pama'][i]

    #                 self.data['porcentaje_solidos_efluente'][i] = 0.04 / 100
    #                 self.data['porcentaje_solidos_lodo'][i] = 0.08
    #                 self.data['porcentaje_solidos_licor'][i] = 0.1 / 100
    #                 self.data['porcentaje_grasa_licor'][i] = self.data['porcentaje_solidos_licor'][i] / 2
    #                 self.data['porcentaje_humedad_ksa'][i] = 0.75
                    
    #                 self.data['porcentaje_grasa_efluente'][i] = self.data['porcentaje_solidos_efluente'][i] / 2
    #                 self.data['flujo_lodo'][i] = self.data['flujo_ic_corregido_dosificacion'][i] * (self.data['solidos_ab_ic_corregido_pama'][i] * 0.000001 - self.data['porcentaje_solidos_efluente'][i]) / (self.data['porcentaje_solidos_lodo'][i] - self.data['porcentaje_solidos_efluente'][i])

    #                 self.data['flujo_efluente'][i] = self.data['flujo_ic_corregido_dosificacion'][i] - self.data['flujo_lodo'][i]
    #                 self.data['porcentaje_grasa_lodo'][i] = (self.data['flujo_ic_corregido_dosificacion'][i] * self.data['porcentaje_grasa_ingreso_clarificador'][i] / 1000000 - self.data['flujo_efluente'][i] * self.data['porcentaje_grasa_efluente'][i]) / self.data['flujo_lodo'][i]

    #                 self.data['flujo_ksa'][i] = self.data['flujo_lodo'][i] * ((1 - self.data['porcentaje_grasa_lodo'][i] - self.data['porcentaje_solidos_lodo'][i]) - (1 - self.data['porcentaje_grasa_licor'][i] - self.data['porcentaje_solidos_licor'][i]))/(self.data['porcentaje_humedad_ksa'][i] - (1 - self.data['porcentaje_grasa_licor'][i] - self.data['porcentaje_solidos_licor'][i]))
    #                 self.data['flujo_licor'][i] = self.data['flujo_lodo'][i] - self.data['flujo_ksa'][i]

    #                 self.data['porcentaje_grasa_ksa'][i] = (self.data['flujo_lodo'][i] * self.data['porcentaje_grasa_lodo'][i] - self.data['flujo_licor'][i] * self.data['porcentaje_solidos_licor'][i]) / self.data['flujo_ksa'][i]
    #                 self.data['porcentaje_solidos_ksa'][i] = 1 - self.data['porcentaje_grasa_ksa'][i] - self.data['porcentaje_humedad_ksa'][i]

    #                 # KK PRENSA
    #                 kk_prensa_tn = mp_ic_tn * aportante_kp / 100
                    
    #                 if i < len(self.data['kk_prensa_tn']):
    #                     self.data['kk_prensa_tn'][i] = kk_prensa_tn
                    
    #                 if i < len(self.data['kk_prensa_hum_tn']):
    #                     self.data['kk_prensa_hum_tn'][i] = kk_prensa_tn * self._parametros.KK_PRENSA_HUMEDAD
                    
    #                 if i < len(self.data['kk_prensa_cen_tn']):
    #                     self.data['kk_prensa_cen_tn'][i] = kk_prensa_tn * self._parametros.KK_PRENSA_CENIZA
                    
    #                 if i < len(self.data['kk_prensa_gra_tn']):
    #                     self.data['kk_prensa_gra_tn'][i] = kk_prensa_tn * self._parametros.KK_PRENSA_GRASA
                    
    #                 if i < len(self.data['kk_prensa_pro_tn']):
    #                     self.data['kk_prensa_pro_tn'][i] = kk_prensa_tn * self._parametros.KK_PRENSA_PROTEINA
                    
    #                 # Porcentajes KK Prensa
    #                 if i < len(self.data['kk_prensa_porc_humedad']):
    #                     self.data['kk_prensa_porc_humedad'][i] = self._parametros.KK_PRENSA_HUMEDAD * 100
                    
    #                 if i < len(self.data['kk_prensa_porc_ceniza']):
    #                     self.data['kk_prensa_porc_ceniza'][i] = self._parametros.KK_PRENSA_CENIZA * 100
                    
    #                 if i < len(self.data['kk_prensa_porc_grasa']):
    #                     self.data['kk_prensa_porc_grasa'][i] = self._parametros.KK_PRENSA_GRASA * 100
                    
    #                 if i < len(self.data['kk_prensa_porc_proteina']):
    #                     self.data['kk_prensa_porc_proteina'][i] = self._parametros.KK_PRENSA_PROTEINA * 100
                    
    #                 # KK SEPARADORA
    #                 kk_separadora_tn = mp_ic_tn * aportante_ks / 100
                    
    #                 if i < len(self.data['kk_separadora_tn']):
    #                     self.data['kk_separadora_tn'][i] = kk_separadora_tn
                    
    #                 if i < len(self.data['kk_sep_hum_tn']):
    #                     self.data['kk_sep_hum_tn'][i] = kk_separadora_tn * self._parametros.KK_SEP_HUMEDAD
                    
    #                 if i < len(self.data['kk_sep_cen_tn']):
    #                     self.data['kk_sep_cen_tn'][i] = kk_separadora_tn * self._parametros.KK_SEP_CENIZA
                    
    #                 if i < len(self.data['kk_sep_gra_tn']):
    #                     self.data['kk_sep_gra_tn'][i] = kk_separadora_tn * self._parametros.KK_SEP_GRASA
                    
    #                 if i < len(self.data['kk_sep_pro_tn']):
    #                     self.data['kk_sep_pro_tn'][i] = kk_separadora_tn * self._parametros.KK_SEP_PROTEINA
                    
    #                 # Porcentajes KK Separadora
    #                 if i < len(self.data['kk_separadora_porc_humedad']):
    #                     self.data['kk_separadora_porc_humedad'][i] = self._parametros.KK_SEP_HUMEDAD * 100
                    
    #                 if i < len(self.data['kk_separadora_porc_ceniza']):
    #                     self.data['kk_separadora_porc_ceniza'][i] = self._parametros.KK_SEP_CENIZA * 100
                    
    #                 if i < len(self.data['kk_separadora_porc_grasa']):
    #                     self.data['kk_separadora_porc_grasa'][i] = self._parametros.KK_SEP_GRASA * 100
                    
    #                 if i < len(self.data['kk_separadora_porc_proteina']):
    #                     self.data['kk_separadora_porc_proteina'][i] = self._parametros.KK_SEP_PROTEINA * 100
                    
    #                 # KK CC TODO: Tener cuidado con los valores en porcentajes
    #                 kk_cc_tn = mp_ic_tn * aportante_cc / 100
                    
    #                 if i < len(self.data['kk_cc_tn']):
    #                     self.data['kk_cc_tn'][i] = kk_cc_tn

    #                 if i < len(self.data['kk_cc_cen_tn']):
    #                     self.data['kk_cc_cen_tn'][i] = self.data['mp_ic_cen_tn'][i] - self.data['kk_prensa_cen_tn'][i] - self.data['kk_sep_cen_tn'][i]

    #                 if i < len(self.data['kk_cc_gra_tn']):
    #                     self.data['kk_cc_gra_tn'][i] = self._parametros.KK_CC_GRASA * kk_cc_tn

    #                 if i < len(self.data['kk_cc_pro_tn']):
    #                     self.data['kk_cc_pro_tn'][i] = (mp_ic_tn - self.data['mp_ic_cen_tn'][i] - self.data['mp_ic_hum_tn'][i] - self.data['mp_ic_gra_tn'][i]) - self.data['kk_prensa_pro_tn'][i] - self.data['kk_sep_pro_tn'][i]
                    
    #                 if i < len(self.data['kk_cc_hum_tn']):
    #                     self.data['kk_cc_hum_tn'][i] = kk_cc_tn - self.data['kk_cc_cen_tn'][i] - self.data['kk_cc_gra_tn'][i] - self.data['kk_cc_pro_tn'][i]

    #                 if i < len(self.data['kk_cc_porc_humedad_2']):
    #                     self.data['kk_cc_porc_humedad_2'][i] = self.data['kk_cc_hum_tn'][i] / kk_cc_tn
    #                     self.data['kk_cc_porc_humedad'][i] = self._parametros.KK_CC_HUMEDAD

    #                 if i < len(self.data['kk_cc_porc_ceniza_2']):
    #                     self.data['kk_cc_porc_ceniza_2'][i] = self.data['kk_cc_cen_tn'][i] / kk_cc_tn
    #                     self.data['kk_cc_porc_ceniza'][i] = self._parametros.KK_CC_CENIZA  

    #                 if i < len(self.data['kk_cc_porc_grasa']):
    #                     self.data['kk_cc_porc_grasa'][i] = self._parametros.KK_CC_GRASA 

    #                 if i < len(self.data['kk_cc_porc_proteina']):
    #                     self.data['kk_cc_porc_proteina'][i] = self._parametros.KK_CC_PROTEINA  

    #                 if i < len(self.data['ksa_porc_humedad']):
    #                     self.data['ksa_porc_humedad'][i] = self._parametros.KSA_HUMEDAD   

    #                 if i < len(self.data['ksa_porc_ceniza']):
    #                     self.data['ksa_porc_ceniza'][i] = self._parametros.KSA_CENIZA

    #                 if i < len(self.data['ksa_porc_grasa']):
    #                     self.data['ksa_porc_grasa'][i] = self.data['porcentaje_grasa_ksa'][i] 

    #                 if i < len(self.data['ksa_porc_proteina']):
    #                     self.data['ksa_porc_proteina'][i] = 1 - self.data['ksa_porc_grasa'][i] - self.data['ksa_porc_humedad'][i] - self.data['ksa_porc_ceniza'][i]

    #                 if i < len(self.data['ksa_gra_tn']):
    #                     self.data['ksa_gra_tn'][i] = self.data['ksa_porc_grasa'][i] * self.data['receta_corregida_dosificacion'][i] * mp_ic_tn

    #                 if i < len(self.data['ksa_pro_tn']):
    #                     self.data['ksa_pro_tn'][i] = self.data['ksa_porc_proteina'][i] * self.data['receta_corregida_dosificacion'][i] * mp_ic_tn

    #                 if i < len(self.data['porcentaje_dosificacion_ksa2']):
    #                     self.data['porcentaje_dosificacion_ksa2'][i] = self.data['receta_corregida_dosificacion'][i]
                  
                    
    #                 # KSA (diferencia)
    #                 ksa_tn = mp_ic_tn - kk_prensa_tn - kk_separadora_tn - kk_cc_tn
                    
    #                 if i < len(self.data['ksa_tn']):
    #                     self.data['ksa_tn'][i] = ksa_tn
                    
    #                 if i < len(self.data['ksa_hum_tn']):
    #                     self.data['ksa_hum_tn'][i] = ksa_tn * self._parametros.KSA_HUMEDAD
                    
    #                 if i < len(self.data['ksa_cen_tn']):
    #                     self.data['ksa_cen_tn'][i] = ksa_tn * self._parametros.KSA_CENIZA
                    
    #                 # KK INTEGRAL
    #                 kk_integral_tn = kk_prensa_tn + kk_separadora_tn + kk_cc_tn + ksa_tn
                    
    #                 if i < len(self.data['kk_integral_tn']):
    #                     self.data['kk_integral_tn'][i] = kk_integral_tn

    #                 if i < len(self.data['kk_integral_hum_tn']):
    #                     self.data['kk_integral_hum_tn'][i] = self.data['kk_prensa_hum_tn'][i] + self.data['kk_sep_hum_tn'][i] + self.data['kk_cc_hum_tn'][i] + self.data['ksa_hum_tn'][i]

    #                 if i < len(self.data['kk_integral_cen_tn']):
    #                     self.data['kk_integral_cen_tn'][i] = self.data['kk_prensa_cen_tn'][i] + self.data['kk_sep_cen_tn'][i] + self.data['kk_cc_cen_tn'][i] + self.data['ksa_cen_tn'][i]

    #                 if i < len(self.data['kk_integral_gra_tn']):
    #                     self.data['kk_integral_gra_tn'][i] = self.data['kk_prensa_gra_tn'][i] + self.data['kk_sep_gra_tn'][i] + self.data['kk_cc_gra_tn'][i] + self.data['ksa_gra_tn'][i]

    #                 if i < len(self.data['kk_integral_pro_tn']):
    #                     self.data['kk_integral_pro_tn'][i] = self.data['kk_prensa_pro_tn'][i] + self.data['kk_sep_pro_tn'][i] + self.data['kk_cc_pro_tn'][i] + self.data['ksa_pro_tn'][i]                        

    #                 if i < len(self.data['kk_integral_porc_humedad']):
    #                     self.data['kk_integral_porc_humedad'][i] = (self.data['kk_integral_hum_tn'][i] / kk_integral_tn)

    #                 if i < len(self.data['kk_integral_porc_ceniza']):
    #                     self.data['kk_integral_porc_ceniza'][i] = (self.data['kk_integral_cen_tn'][i] / kk_integral_tn)
                    
    #                 if i < len(self.data['kk_integral_porc_grasa']):
    #                     self.data['kk_integral_porc_grasa'][i] = (self.data['kk_integral_gra_tn'][i] / kk_integral_tn)
                    
    #                 if i < len(self.data['kk_integral_porc_proteina']):
    #                     self.data['kk_integral_porc_proteina'][i] = (self.data['kk_integral_pro_tn'][i] / kk_integral_tn)

    #                 if i < len(self.data['harina_porc_humedad_2']):
    #                     self.data['harina_porc_humedad_2'][i] = self._parametros.HARINA_HUMEDAD

    #                 if i < len(self.data['harina_porc_ceniza_2']):
    #                     self.data['harina_porc_ceniza_2'][i] = self.data['kk_integral_porc_ceniza'][i]/((self.data['kk_integral_porc_ceniza'][i]+self.data['kk_integral_porc_grasa'][i]+self.data['kk_integral_porc_proteina'][i])/(100-self.data['harina_porc_humedad_2'][i]))

    #                 if i < len(self.data['harina_porc_grasa_2']):
    #                     self.data['harina_porc_grasa_2'][i] = self.data['kk_integral_porc_grasa'][i]/((self.data['kk_integral_porc_ceniza'][i]+self.data['kk_integral_porc_grasa'][i]+self.data['kk_integral_porc_proteina'][i])/(100-self.data['harina_porc_humedad_2'][i]))

    #                 if i < len(self.data['harina_porc_proteina_2']):
    #                     self.data['harina_porc_proteina_2'][i] = self.data['kk_integral_porc_proteina'][i]/((self.data['kk_integral_porc_ceniza'][i]+self.data['kk_integral_porc_grasa'][i]+self.data['kk_integral_porc_proteina'][i])/(100-self.data['harina_porc_humedad_2'][i]))
                        
    #         except (TypeError, ValueError, ZeroDivisionError) as e:
    #             continue

    def calcular_vectores_productos(self):
        """Calcula vectores de productos de manera segura"""
        longitud = self._longitud_principal
        if longitud == 0:
            return
        
        for i in range(longitud):
            try:
                if (i < len(self.data['mp_ic_tn']) and 
                    not pd.isna(self.data['mp_ic_tn'][i]) and 
                    self.data['mp_ic_tn'][i] > 0 and
                    i < len(self.data['aportante_porc_kp']) and
                    i < len(self.data['aportante_porc_ks']) and
                    i < len(self.data['aportante_porc_cc'])):
                    
                    mp_ic_tn = float(self.data['mp_ic_tn'][i])
                    aportante_kp = float(self.data['aportante_porc_kp'][i])
                    aportante_ks = float(self.data['aportante_porc_ks'][i])
                    aportante_cc = float(self.data['aportante_porc_cc'][i])
                    
                    # 1️⃣ Obtener calidad
                    calidad = None
                    if 'calidad_harina' in self.data and i < len(self.data['calidad_harina']):
                        calidad = self.data['calidad_harina'][i]
                    if pd.isna(calidad) or calidad not in ['A', 'B', 'C', 'D']:
                        calidad = 'A'  # valor por defecto
                        
                    # ------------------------------------------------------------
                    # 2️⃣ Obtener parámetros dinámicos según la calidad
                    prensa_hum = self._parametros.KK_PRENSA_HUMEDAD.get(calidad, 0)
                    prensa_cen = self._parametros.KK_PRENSA_CENIZA.get(calidad, 0)
                    prensa_gra = self._parametros.KK_PRENSA_GRASA.get(calidad, 0)
                    prensa_pro = self._parametros.KK_PRENSA_PROTEINA.get(calidad, 0)
                    
                    # 2️⃣ Obtener parámetros dinámicos según la calidad
                    sep_hum = self._parametros.KK_SEP_HUMEDAD.get(calidad, 0)
                    sep_cen = self._parametros.KK_SEP_CENIZA.get(calidad, 0)
                    sep_gra = self._parametros.KK_SEP_GRASA.get(calidad, 0)
                    sep_pro = self._parametros.KK_SEP_PROTEINA.get(calidad, 0)
                    
                    # 2️⃣ Obtener parámetros dinámicos según la calidad
                    conc_hum = self._parametros.KK_CC_HUMEDAD.get(calidad, 0)
                    conc_cen = self._parametros.KK_CC_CENIZA.get(calidad, 0)
                    conc_gra = self._parametros.KK_CC_GRASA.get(calidad, 0)
                    conc_pro = self._parametros.KK_CC_PROTEINA.get(calidad, 0)
                    
                    # 2️⃣ Obtener parámetros dinámicos según la calidad
                    ksa_hum = self._parametros.KSA_HUMEDAD.get(calidad, 0)
                    ksa_cen = self._parametros.KSA_CENIZA.get(calidad, 0)
                    
                    # VECTORES PREVIOS TODO:
                    if i < len(self.data['porcentaje_grasa_ingreso_clarificador']):
                        self.data['porcentaje_grasa_ingreso_clarificador'][i] = 0.5 * self.data['grasa_ab_corregido_pama'][i]

                    self.data['porcentaje_solidos_efluente'][i] = 0.04 / 100
                    self.data['porcentaje_solidos_lodo'][i] = 0.08
                    self.data['porcentaje_solidos_licor'][i] = 0.1 / 100
                    self.data['porcentaje_grasa_licor'][i] = self.data['porcentaje_solidos_licor'][i] / 2
                    self.data['porcentaje_humedad_ksa'][i] = 0.75
                    
                    self.data['porcentaje_grasa_efluente'][i] = self.data['porcentaje_solidos_efluente'][i] / 2
                    self.data['flujo_lodo'][i] = self.data['flujo_ic_corregido_dosificacion'][i] * (self.data['solidos_ab_ic_corregido_pama'][i] * 0.000001 - self.data['porcentaje_solidos_efluente'][i]) / (self.data['porcentaje_solidos_lodo'][i] - self.data['porcentaje_solidos_efluente'][i])

                    self.data['flujo_efluente'][i] = self.data['flujo_ic_corregido_dosificacion'][i] - self.data['flujo_lodo'][i]
                    self.data['porcentaje_grasa_lodo'][i] = (self.data['flujo_ic_corregido_dosificacion'][i] * self.data['porcentaje_grasa_ingreso_clarificador'][i] / 1000000 - self.data['flujo_efluente'][i] * self.data['porcentaje_grasa_efluente'][i]) / self.data['flujo_lodo'][i]

                    self.data['flujo_ksa'][i] = self.data['flujo_lodo'][i] * ((1 - self.data['porcentaje_grasa_lodo'][i] - self.data['porcentaje_solidos_lodo'][i]) - (1 - self.data['porcentaje_grasa_licor'][i] - self.data['porcentaje_solidos_licor'][i]))/(self.data['porcentaje_humedad_ksa'][i] - (1 - self.data['porcentaje_grasa_licor'][i] - self.data['porcentaje_solidos_licor'][i]))
                    self.data['flujo_licor'][i] = self.data['flujo_lodo'][i] - self.data['flujo_ksa'][i]

                    self.data['porcentaje_grasa_ksa'][i] = (self.data['flujo_lodo'][i] * self.data['porcentaje_grasa_lodo'][i] - self.data['flujo_licor'][i] * self.data['porcentaje_solidos_licor'][i]) / self.data['flujo_ksa'][i]
                    self.data['porcentaje_solidos_ksa'][i] = 1 - self.data['porcentaje_grasa_ksa'][i] - self.data['porcentaje_humedad_ksa'][i]

                    # KK PRENSA
                    kk_prensa_tn = mp_ic_tn * aportante_kp / 100
                    
                    # ------------------------------------------------------------
                    # 3️⃣ Calcular vectores KK PRENSA usando esos valores
                    kk_prensa_tn = mp_ic_tn * aportante_kp / 100

                    if i < len(self.data['kk_prensa_tn']):
                        self.data['kk_prensa_tn'][i] = kk_prensa_tn

                    if i < len(self.data['kk_prensa_hum_tn']):
                        self.data['kk_prensa_hum_tn'][i] = kk_prensa_tn * prensa_hum

                    if i < len(self.data['kk_prensa_cen_tn']):
                        self.data['kk_prensa_cen_tn'][i] = kk_prensa_tn * prensa_cen

                    if i < len(self.data['kk_prensa_gra_tn']):
                        self.data['kk_prensa_gra_tn'][i] = kk_prensa_tn * prensa_gra

                    if i < len(self.data['kk_prensa_pro_tn']):
                        self.data['kk_prensa_pro_tn'][i] = kk_prensa_tn * prensa_pro
                    
                    # ------------------------------------------------------------
                    # 4️⃣ Porcentajes
                    if i < len(self.data['kk_prensa_porc_humedad']):
                        self.data['kk_prensa_porc_humedad'][i] = prensa_hum * 100

                    if i < len(self.data['kk_prensa_porc_ceniza']):
                        self.data['kk_prensa_porc_ceniza'][i] = prensa_cen * 100

                    if i < len(self.data['kk_prensa_porc_grasa']):
                        self.data['kk_prensa_porc_grasa'][i] = prensa_gra * 100

                    if i < len(self.data['kk_prensa_porc_proteina']):
                        self.data['kk_prensa_porc_proteina'][i] = prensa_pro * 100
                    
                    # KK SEPARADORA
                    kk_separadora_tn = mp_ic_tn * aportante_ks / 100
                    
                    if i < len(self.data['kk_separadora_tn']):
                        self.data['kk_separadora_tn'][i] = kk_separadora_tn
                    
                    if i < len(self.data['kk_sep_hum_tn']):
                        self.data['kk_sep_hum_tn'][i] = kk_separadora_tn * sep_hum
                    
                    if i < len(self.data['kk_sep_cen_tn']):
                        self.data['kk_sep_cen_tn'][i] = kk_separadora_tn * sep_cen
                    
                    if i < len(self.data['kk_sep_gra_tn']):
                        self.data['kk_sep_gra_tn'][i] = kk_separadora_tn * sep_gra
                    
                    if i < len(self.data['kk_sep_pro_tn']):
                        self.data['kk_sep_pro_tn'][i] = kk_separadora_tn * sep_pro
                    
                    # Porcentajes KK Separadora
                    if i < len(self.data['kk_separadora_porc_humedad']):
                        self.data['kk_separadora_porc_humedad'][i] = sep_hum * 100
                    
                    if i < len(self.data['kk_separadora_porc_ceniza']):
                        self.data['kk_separadora_porc_ceniza'][i] = sep_cen * 100
                    
                    if i < len(self.data['kk_separadora_porc_grasa']):
                        self.data['kk_separadora_porc_grasa'][i] = sep_gra * 100
                    
                    if i < len(self.data['kk_separadora_porc_proteina']):
                        self.data['kk_separadora_porc_proteina'][i] = sep_pro * 100
                    
                    # KK CC TODO: Tener cuidado con los valores en porcentajes
                    kk_cc_tn = mp_ic_tn * aportante_cc / 100
                    
                    if i < len(self.data['kk_cc_tn']):
                        self.data['kk_cc_tn'][i] = kk_cc_tn

                    if i < len(self.data['kk_cc_cen_tn']):
                        self.data['kk_cc_cen_tn'][i] = self.data['mp_ic_cen_tn'][i] - self.data['kk_prensa_cen_tn'][i] - self.data['kk_sep_cen_tn'][i]

                    if i < len(self.data['kk_cc_gra_tn']):
                        self.data['kk_cc_gra_tn'][i] = conc_gra * kk_cc_tn

                    if i < len(self.data['kk_cc_pro_tn']):
                        self.data['kk_cc_pro_tn'][i] = (mp_ic_tn - self.data['mp_ic_cen_tn'][i] - self.data['mp_ic_hum_tn'][i] - self.data['mp_ic_gra_tn'][i]) - self.data['kk_prensa_pro_tn'][i] - self.data['kk_sep_pro_tn'][i]
                    
                    if i < len(self.data['kk_cc_hum_tn']):
                        self.data['kk_cc_hum_tn'][i] = kk_cc_tn - self.data['kk_cc_cen_tn'][i] - self.data['kk_cc_gra_tn'][i] - self.data['kk_cc_pro_tn'][i]

                    if i < len(self.data['kk_cc_porc_humedad_2']):
                        self.data['kk_cc_porc_humedad_2'][i] = self.data['kk_cc_hum_tn'][i] / kk_cc_tn
                        self.data['kk_cc_porc_humedad'][i] = conc_hum

                    if i < len(self.data['kk_cc_porc_ceniza_2']):
                        self.data['kk_cc_porc_ceniza_2'][i] = self.data['kk_cc_cen_tn'][i] / kk_cc_tn
                        self.data['kk_cc_porc_ceniza'][i] = conc_cen

                    if i < len(self.data['kk_cc_porc_grasa']):
                        self.data['kk_cc_porc_grasa'][i] = conc_gra

                    if i < len(self.data['kk_cc_porc_proteina']):
                        self.data['kk_cc_porc_proteina'][i] = conc_pro

                    if i < len(self.data['ksa_porc_humedad']):
                        self.data['ksa_porc_humedad'][i] = ksa_hum

                    if i < len(self.data['ksa_porc_ceniza']):
                        self.data['ksa_porc_ceniza'][i] = ksa_cen

                    if i < len(self.data['ksa_porc_grasa']):
                        self.data['ksa_porc_grasa'][i] = self.data['porcentaje_grasa_ksa'][i] 

                    if i < len(self.data['ksa_porc_proteina']):
                        self.data['ksa_porc_proteina'][i] = 1 - self.data['ksa_porc_grasa'][i] - self.data['ksa_porc_humedad'][i] - self.data['ksa_porc_ceniza'][i]

                    if i < len(self.data['ksa_gra_tn']):
                        self.data['ksa_gra_tn'][i] = self.data['ksa_porc_grasa'][i] * self.data['receta_corregida_dosificacion'][i] * mp_ic_tn

                    if i < len(self.data['ksa_pro_tn']):
                        self.data['ksa_pro_tn'][i] = self.data['ksa_porc_proteina'][i] * self.data['receta_corregida_dosificacion'][i] * mp_ic_tn

                    if i < len(self.data['porcentaje_dosificacion_ksa2']):
                        self.data['porcentaje_dosificacion_ksa2'][i] = self.data['receta_corregida_dosificacion'][i]
                  
                    
                    # KSA (diferencia)
                    ksa_tn = mp_ic_tn - kk_prensa_tn - kk_separadora_tn - kk_cc_tn
                    
                    if i < len(self.data['ksa_tn']):
                        self.data['ksa_tn'][i] = ksa_tn
                    
                    if i < len(self.data['ksa_hum_tn']):
                        self.data['ksa_hum_tn'][i] = ksa_tn * ksa_hum
                    
                    if i < len(self.data['ksa_cen_tn']):
                        self.data['ksa_cen_tn'][i] = ksa_tn * ksa_cen
                    
                    # KK INTEGRAL
                    kk_integral_tn = kk_prensa_tn + kk_separadora_tn + kk_cc_tn + ksa_tn
                    
                    if i < len(self.data['kk_integral_tn']):
                        self.data['kk_integral_tn'][i] = kk_integral_tn

                    if i < len(self.data['kk_integral_hum_tn']):
                        self.data['kk_integral_hum_tn'][i] = self.data['kk_prensa_hum_tn'][i] + self.data['kk_sep_hum_tn'][i] + self.data['kk_cc_hum_tn'][i] + self.data['ksa_hum_tn'][i]

                    if i < len(self.data['kk_integral_cen_tn']):
                        self.data['kk_integral_cen_tn'][i] = self.data['kk_prensa_cen_tn'][i] + self.data['kk_sep_cen_tn'][i] + self.data['kk_cc_cen_tn'][i] + self.data['ksa_cen_tn'][i]

                    if i < len(self.data['kk_integral_gra_tn']):
                        self.data['kk_integral_gra_tn'][i] = self.data['kk_prensa_gra_tn'][i] + self.data['kk_sep_gra_tn'][i] + self.data['kk_cc_gra_tn'][i] + self.data['ksa_gra_tn'][i]

                    if i < len(self.data['kk_integral_pro_tn']):
                        self.data['kk_integral_pro_tn'][i] = self.data['kk_prensa_pro_tn'][i] + self.data['kk_sep_pro_tn'][i] + self.data['kk_cc_pro_tn'][i] + self.data['ksa_pro_tn'][i]                        

                    if i < len(self.data['kk_integral_porc_humedad']):
                        self.data['kk_integral_porc_humedad'][i] = (self.data['kk_integral_hum_tn'][i] / kk_integral_tn)

                    if i < len(self.data['kk_integral_porc_ceniza']):
                        self.data['kk_integral_porc_ceniza'][i] = (self.data['kk_integral_cen_tn'][i] / kk_integral_tn)
                    
                    if i < len(self.data['kk_integral_porc_grasa']):
                        self.data['kk_integral_porc_grasa'][i] = (self.data['kk_integral_gra_tn'][i] / kk_integral_tn)
                    
                    if i < len(self.data['kk_integral_porc_proteina']):
                        self.data['kk_integral_porc_proteina'][i] = (self.data['kk_integral_pro_tn'][i] / kk_integral_tn)

                    if i < len(self.data['harina_porc_humedad_2']):
                        self.data['harina_porc_humedad_2'][i] = self._parametros.HARINA_HUMEDAD

                    if i < len(self.data['harina_porc_ceniza_2']):
                        self.data['harina_porc_ceniza_2'][i] = self.data['kk_integral_porc_ceniza'][i]/((self.data['kk_integral_porc_ceniza'][i]+self.data['kk_integral_porc_grasa'][i]+self.data['kk_integral_porc_proteina'][i])/(100-self.data['harina_porc_humedad_2'][i]))

                    if i < len(self.data['harina_porc_grasa_2']):
                        self.data['harina_porc_grasa_2'][i] = self.data['kk_integral_porc_grasa'][i]/((self.data['kk_integral_porc_ceniza'][i]+self.data['kk_integral_porc_grasa'][i]+self.data['kk_integral_porc_proteina'][i])/(100-self.data['harina_porc_humedad_2'][i]))

                    if i < len(self.data['harina_porc_proteina_2']):
                        self.data['harina_porc_proteina_2'][i] = self.data['kk_integral_porc_proteina'][i]/((self.data['kk_integral_porc_ceniza'][i]+self.data['kk_integral_porc_grasa'][i]+self.data['kk_integral_porc_proteina'][i])/(100-self.data['harina_porc_humedad_2'][i]))
                        
            except (TypeError, ValueError, ZeroDivisionError) as e:
                continue


    def calcular_vectores_resultados_finales(self):
        """Calcula los resultados finales del proceso"""
        longitud = self._longitud_principal
        if longitud == 0:
            return
        
        for i in range(longitud):
            try:
                # HAR TON
                if (i < len(self.data['kk_integral_cen_tn']) and 
                    i < len(self.data['kk_integral_gra_tn']) and 
                    i < len(self.data['kk_integral_pro_tn']) and
                    i < len(self.data['harina_porc_humedad']) and
                    not pd.isna(self.data['kk_integral_cen_tn'][i]) and
                    not pd.isna(self.data['kk_integral_gra_tn'][i]) and
                    not pd.isna(self.data['kk_integral_pro_tn'][i]) and
                    not pd.isna(self.data['harina_porc_humedad'][i])):
                    
                    componentes_secos = (self.data['kk_integral_cen_tn'][i] + 
                                       self.data['kk_integral_gra_tn'][i] + 
                                       self.data['kk_integral_pro_tn'][i])
                    
                    har_ton = componentes_secos / (1 - self.data['harina_porc_humedad'][i] / 100)
                    
                    if i < len(self.data['har_ton']):
                        self.data['har_ton'][i] = har_ton

                    aceite_tn = self.data['mp_ic_gra_tn'][i] - (self.data['kk_prensa_gra_tn'][i]+self.data['kk_sep_gra_tn'][i] + self.data['kk_cc_gra_tn'][i])
                    if i < len(self.data['aceite_tn']):
                        self.data['aceite_tn'][i] = aceite_tn

                # RENDIMIENTO HARINA
                if (i < len(self.data['mp_ic_tn']) and 
                    i < len(self.data['har_ton']) and
                    not pd.isna(self.data['mp_ic_tn'][i]) and 
                    not pd.isna(self.data['har_ton'][i]) and
                    self.data['mp_ic_tn'][i] > 0):
                    
                    rendimiento = self.data['mp_ic_tn'][i] / self.data['har_ton'][i]
                    
                    if i < len(self.data['rendimiento_harina']):
                        self.data['rendimiento_harina'][i] = rendimiento

                    rendimiento_aceite = self.data['aceite_tn'][i] / self.data['mp_ic_tn'][i]
                    if i < len(self.data['rendimiento_aceite']):
                        self.data['rendimiento_aceite'][i] = rendimiento_aceite
                        
            except (TypeError, ValueError, ZeroDivisionError) as e:
                continue

    def calcular_vectores_clarificador(self):
        """Calcula vectores del clarificador (placeholder)"""
        # Implementación simplificada - se puede expandir luego
        pass

    def obtener_dataframe_completo(self) -> pd.DataFrame:
        """Retorna DataFrame con todas las columnas"""
        if self._longitud_principal == 0:
            return pd.DataFrame()
        
        longitudes = [len(vector) for vector in self.data.values()]
        longitud_maxima = max(longitudes) if longitudes else 0
        
        df_data = {}
        for nombre_vector, vector in self.data.items():
            if len(vector) < longitud_maxima:
                if vector.dtype == object:
                    extension = np.array([None] * (longitud_maxima - len(vector)), dtype=object)
                else:
                    extension = np.full(longitud_maxima - len(vector), np.nan, dtype=vector.dtype)
                
                vector_extendido = np.concatenate([vector, extension])
                df_data[nombre_vector] = vector_extendido
            else:
                df_data[nombre_vector] = vector
        
        df = pd.DataFrame(df_data)
        
        for col in df.columns:
            if df[col].dtype == object:
                df[col] = df[col].where(pd.notna(df[col]), None)
        
        return df

    def ejecutar_calculos_completos(self):
        """Ejecuta todos los cálculos del proceso completo"""
        try:
            print("🔧 Inicializando vectores...")
            self.inicializar_vectores_proceso()
            
            print("📊 Simulando progreso...")
            self.simular_progreso_completo()
            
            print("🌱 Calculando materia prima...")
            self.calcular_vectores_materia_prima()
            
            print("⚙️ Calculando proceso...")
            self.calcular_vectores_proceso()
            
            print("📈 Calculando flujos...")
            self.calcular_vectores_flujos()
            
            print("🏭 Calculando productos...")
            self.calcular_vectores_productos()
            
            print("📊 Calculando resultados finales...")
            self.calcular_vectores_resultados_finales()
            
            print("✅ Todos los cálculos completados exitosamente")
            return self.data
            
        except Exception as e:
            print(f"❌ Error durante los cálculos: {e}")
            import traceback
            traceback.print_exc()
            return self.data
        
def cargar_datos_plan_descarga_y_tablas_mio(id_pd):
    query_pd = f"""
    SELECT A.*, B.id_planta, 
    B.FlagPPyPZ,
    case when TE = 'Propio C/Frio' then 'RC'
    when TE = 'Propio S/Frio' then 'SF'
    else 'SF' end as TIPO,
    0.1 AS JuvenilesPonderado
    FROM PlanDescargaOutput A
    LEFT JOIN PlanDescargaModeloLog B
    ON A.IDLog=B.id
    WHERE B.FlagPPyPZ = 1 AND B.id = {id_pd};"""
    discharge_plan_df_original = process_discharge_plan_data(pd.read_sql(query_pd, cnxn_mio))

    terceros_frios = [
        "RICARDO",
        "OLGA",
        "PATRICIA",
        "ALESSANDRO",
        "CHAVELI II",
        "ISABELITA",
        "STEFANO",
        "DON LUCHO",
        "DANIELLA",
        "SEBASTIAN",
        "BAMAR II",
        "MODESTO 3",
        "MODESTO 6",
    ]
    mask = discharge_plan_df_original["Embarcacion"].isin(terceros_frios)
    discharge_plan_df_original.loc[mask, "TIPO"] = 'RC'

    query = """
    select  top 60 Marea,FECHA ,TRIM(PLANTA) PLANTA , TVN_DES ,DECLARA, DESCARG ,GRA_DESCAR , HUM_DESCAR 
    from sap.ZQM_DESC_MP_CHI2 zdmc2 
    where F_ARRIB  >cast('2024-04-21' as date) AND DESCARG >0 and GRA_DESCAR > 0
    order by Marea desc;
    """

    grasa_df_original = pd.read_sql(query, cnxn_juv)
    grasa_df = grasa_df_original.copy()
    grasa_df["PLANTA"] = grasa_df["PLANTA"].str.upper()
    grasa_df = grasa_df.groupby('PLANTA', as_index=False)["GRA_DESCAR"].mean()

    to_replace = {
        "MALABRIGO SUR": "MALABRIGO",
        "CALLAO NORTE": "CALLAO",
    }
    grasa_df["PLANTA"] = grasa_df["PLANTA"].replace(to_replace)

    query = """
    SELECT * FROM dbo.Parametros_proceso
    """
    limpieza_tuberia_df_orig = pd.read_sql(query, cnxn_mio)

    query = """
    SELECT * FROM dbo.Chatas_Lineas
    """
    chatas_lineas_df_orig = pd.read_sql(query, cnxn_mio)

    query = """
    SELECT * FROM dbo.Tipo_Descarga
    """
    condiciones_rap_orig = pd.read_sql(query, cnxn_mio)

    calidad_ic_df = limpieza_tuberia_df_orig[["PLANTA", "UmbralIC_TVN"]].copy()

    query = """
    SELECT * FROM dbo.Tabla_velocidades
    """
    velocidades_volumen_df_orig = pd.read_sql(query, cnxn_mio)

    query = """
    SELECT * FROM dbo.Dosificacion_KSA_calidades"""
    UmbralTVN_Calidades = process_calidades(pd.read_sql(query, cnxn_mio))

    query = """
    SELECT * FROM dbo.KK_calidades"""
    ValoresFijosPP_orig = pd.read_sql(query, cnxn_mio)

    query = """
    SELECT * FROM dbo.Balance_KSA"""
    balance_ksa_df_orig = pd.read_sql(query, cnxn_mio)

    query = """
    SELECT * FROM dbo.FECHA_AJUSTADO_PP"""
    hora_arranque_df = pd.read_sql(query, cnxn_mio)

    now = datetime.now()
    discharge_plan_df = discharge_plan_df_original.copy()
    fecha_batch = discharge_plan_df["Fecha"].unique()[0]
    try:
        planta_selected = discharge_plan_df["id_planta"].unique()[0]
    except:
        pass
    try:
        grasa_h75 = grasa_df[grasa_df["PLANTA"] == planta_selected]["GRA_DESCAR"].values[0]
    except:
        grasa_h75 = grasa_df["GRA_DESCAR"].mean()
        
    print("Grasa H75", grasa_h75)
    limpieza_tuberia_df = process_limpieza_tuberia_data(limpieza_tuberia_df_orig, planta_selected)
    limite_base_seca = limpieza_tuberia_df["LimiteCapacidadSeparadoraAmbiental"].values[0]
    print("Limite Base Seca", limite_base_seca)
    limite_flujo_ic = limpieza_tuberia_df["ValorFlujoIngreso"].values[0]
    print("Limite Flujo IC", limite_flujo_ic)
    chatas_lineas_df = process_chatas_lineas_data(chatas_lineas_df_orig, planta_selected)
    condiciones_rap = process_condiciones_rap(condiciones_rap_orig, planta_selected)
    velocidades_volumen_df = process_velocidades(velocidades_volumen_df_orig, planta_selected)
    balance_ksa_df = process_balance_ksa_data(balance_ksa_df_orig, planta_selected)
    stock_minimo_pama = hallar_volumen_minimo_pama(velocidades_volumen_df, grasa_h75)
    ValoresFijosPP = process_valores_fijos(ValoresFijosPP_orig, planta_selected)
    arranque = process_arranque(hora_arranque_df, planta_selected, fecha_batch)
    volumen_materia_prima = discharge_plan_df["VolumenEstTM"].sum()
    v_init, v_intermedia_inicial, v_cruise, v_intermedia_final, v_term = hallar_velocidades_base(velocidades_volumen_df, grasa_h75, volumen_materia_prima)
    
    # ------------------------------------------Procesamiento de datos ------------------------------------------------------------------#

    # Generación de Intervalos
    intervalos_df = generate_intervalo_horas(discharge_plan_df)

    # Convesion de velocidades a lista
    V_tot     = discharge_plan_df["VolumenEstTM"].sum()    # volumen total a procesar

    velocidades = {
        'Velocidad Inicio': v_init,
        'Velocidad Intermedia1': v_intermedia_inicial,
        'Velocidad Maxima': v_cruise,
        'Velocidad Intermedia2': v_intermedia_final,
        'Velocidad Final': v_term
    }
    # volumen = 260.0

    speeds_fixed, perfil_lbls = generar_perfil_velocidades(V_tot, velocidades)
    # Generación de Estados de Poza
    nombre_velocidad = []
    for ind, vel in enumerate(speeds_fixed):
        cont = 0
        for tipo in velocidades.keys():
            if vel == velocidades[tipo]:
                if tipo == "Velocidad Inicio" and ind > 3:
                    continue
                nombre_velocidad.append(tipo)
                cont = cont + 1
                break
        if cont == 0:
            nombre_velocidad.append("Velocidad Final")

    df_pozas = pd.DataFrame(np.arange(1, pozas+1), columns=["Poza"])
    intervalos_df['key'] = 0
    df_pozas['key'] = 0
    status_pozas = pd.merge(intervalos_df, df_pozas, on='key', how='outer')
    del status_pozas["key"]
    status_pozas["StockActual"] = 0
    status_pozas["TVN_Actual"] = 0
    status_pozas["StockAlimentado"] = 0
    status_pozas["TVNAlimentado"] = 0
    status_pozas["Capacidad_Pozas"] = 300

    # Completar intervalos con la data de descarga
    eps = discharge_plan_df["Embarcacion"].unique()
    mareas_actuales = discharge_plan_df["MareaId"].unique()
    discharge_plan_df["TDC_Descarga"] = discharge_plan_df["TDC_Descarga"].clip(upper=28)
    intervals_filled_df = pd.concat([fill_intervals(intervalos_df, discharge_plan_df[discharge_plan_df["MareaId"] == marea_id]) for marea_id in mareas_actuales])

    # Actualizar TDC a la Descarga
    intervals_filled_df["FlagHora"] = 0
    mask = intervals_filled_df["Duracion"] > 0
    intervals_filled_df.loc[mask, 'FlagHora'] = 1
    intervals_filled_df["to_update_tdc"] = intervals_filled_df.groupby(['Embarcacion'])['FlagHora'].cumsum() - 1
    intervals_filled_df["TDC_Descarga_Actual"] = np.where(intervals_filled_df["Volumen"] > 0, intervals_filled_df["TDC_Descarga_Inicial"] + intervals_filled_df["to_update_tdc"], 0)

    # Agregar el RAP a la descarga
    intervals_rap = add_rap(intervals_filled_df, chatas_lineas_df, condiciones_rap, limpieza_tuberia_df)

    # Calcula el RAP PAMA
    intervals_rap["VolumenRAP"] = intervals_rap["Volumen"] * intervals_rap["RAP"]
    intervals_rap["VolumenTromel"] = intervals_rap["Duracion"] * 8
    intervals_rap["RAP_Limpieza_Tromel"] = intervals_rap["VolumenRAP"] + (intervals_rap["LimpiezaTuberia"] * intervals_rap["Duracion"]) + intervals_rap["VolumenTromel"]
    intervals_rap["Limpieza_de_Tuberia"] = intervals_rap["LimpiezaTuberia"] * intervals_rap["Duracion"]
    intervals_rap["VolumenTotalAB"] = intervals_rap["RAP_Limpieza_Tromel"] * intervals_rap["FactorCorreccion"] / 100
    intervals_rap["RAP_PAMA"] = intervals_rap["VolumenTotalAB"] / intervals_rap["Volumen"]


    intervals_rap["TVNDescarga"] = np.where(intervals_rap["TipoEP"].isin(['SF']), np.vectorize(estimate_tvn_sin_frio)(intervals_rap['TDC_Descarga_Actual']), np.vectorize(estimate_tvn_con_frio)(intervals_rap['TDC_Descarga_Actual']))
    intervals_rap["TVNDescarga"] = np.where(intervals_rap["Volumen"] > 0, intervals_rap["TVNDescarga"], np.nan)
    resultados_pp_grouped = group_intervals(intervals_rap, stock_minimo_pama)
    resultados_pp_grouped["flg_min_acum"] = resultados_pp_grouped["flg_min"].cumsum()
    volumen_ab = resultados_pp_grouped["VolumenPama"].max()
    
    
    
    # Definición de Hora de Arranque

    resultados_pp_grouped["flg_min_acum"] = resultados_pp_grouped["flg_min"].cumsum()
    # try:
    #     if len(arranque) == 0:
    #         index_arranque = resultados_pp_grouped[resultados_pp_grouped['flg_min'] == 1].iloc[0].name
    # except:
    #     timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
    #     logs = pd.DataFrame({
    #         'id_pd': [id],
    #         'StatusOptimizacion': ['0'],
    #         'Mensaje': ['No se alcanzó volumen mínimo de arranque'],
    #         'Timestamp': [timestamp],
    #     })
    #     if ambiente == 'prod':
    #         print("INGRESANDO LOGS PP A SQL")
    #         logs.to_sql('PlanProduccionLog', enginePC, if_exists='append', index=False)
    #     else:
    #         print("No se alcanzó volumen mínimo de arranque")
    #     continue
    
    if len(arranque) > 0:
        idx_min = (resultados_pp_grouped['Intervalo Poza'] - arranque.values[0]).abs().idxmin()
        cook_start_hour = resultados_pp_grouped.loc[idx_min]["NumHora"]
    else:
        cook_start_hour = resultados_pp_grouped.loc[index_arranque, 'NumHora'] + 1

    try:
        index_arranque = resultados_pp_grouped[resultados_pp_grouped['flg_min'] == 1].iloc[0].name
        hora_cook = resultados_pp_grouped.loc[index_arranque, 'NumHora'] + 1
        cook_start_hour_model = resultados_pp_grouped.loc[resultados_pp_grouped["NumHora"] == hora_cook, "Intervalo Poza"].values[0]
    except:   
        cook_start_hour_model = None
    
    descargas = intervals_rap.copy()
    status_init = status_pozas.copy()
    
    
    
    
    # Calculo de pozas
    descargas = intervals_rap.copy()
    status_init = status_pozas.copy()
    
    ## Calculo de descarga, almacenamiento y cocción en pozas
    first_hour = status_init['NumHora'].min()
    initial = (status_init[status_init['NumHora'] == first_hour]
            .dropna(subset=['Poza'])
            .copy())
    initial['Poza'] = initial['Poza'].astype(int)
    initial = initial.set_index('Poza')[['StockActual', 'TVN_Actual', 'Capacidad_Pozas']]
    initial.columns = ['Stock', 'TVN_Actual', 'Capacidad']
    
    # Copias para simulación
    pools = initial.copy()

    # Almacenar resultados
    records_desc = []
    cook_records = []
    
    def sim_descarga_coccion(pools_df, descargas_df, speeds_fixed, cook_start_hour, threshold_degradacion):
        pools = pools_df.copy()
        descargas = descargas_df.copy()
        # Rango total de horas a simular
        min_hora, max_descarga_hora = descargas['NumHora'].min(), descargas['NumHora'].max()
        max_coccion_hora = cook_start_hour + len(speeds_fixed) - 1
        max_hora = max(max_descarga_hora, max_coccion_hora)

        # Índice para speeds_fixed
        cook_idx = 0

        for hora in range(min_hora, max_hora + 1):
            # --- DESCARGA ---
            # Inicializar métricas de descarga para esta hora
            pools['StockAlimentado'] = 0.0
            pools['sum_vol_tvn'] = 0.0
            pools['TVN_Actual'].fillna(0, inplace=True)
            
            # Si hay descargas en esta hora, asignarlas
            for _, row in descargas[descargas['NumHora'] == hora].iterrows():
                remaining, tvn_in = row['Volumen'], estimate_tvn_sin_frio(row['TDC_Descarga_Actual']) if row["TipoEP"] == 'SF' else estimate_tvn_con_frio(row['TDC_Descarga_Actual'])
                while remaining > 0:
                    pools['Disponible'] = pools['Capacidad'] - pools['Stock']
                    avail = pools[pools['Disponible'] > 0]
                    if avail.empty:
                        raise ValueError(f"No hay capacidad disponible en hora {hora}")
                    cand = avail[abs(avail['TVN_Actual'] - tvn_in) <= threshold_degradacion]
                    if cand.empty:
                        cand = avail
                    sel = abs(cand['TVN_Actual'] - tvn_in).idxmin()
                    alloc = min(remaining, pools.at[sel, 'Disponible'])
                    # Actualizar descarga
                    pools.at[sel, 'StockAlimentado'] += alloc
                    pools.at[sel, 'sum_vol_tvn']     += alloc * tvn_in
                    old_s, old_t = pools.at[sel, 'Stock'], pools.at[sel, 'TVN_Actual']
                    new_s = old_s + alloc
                    pools.at[sel, 'Stock'] = new_s
                    pools.at[sel, 'TVN_Actual'] = ((old_t * old_s + tvn_in * alloc) / new_s
                                                if old_s > 0 else tvn_in)
                    remaining -= alloc

            pools['TVNAlimentado'] = pools['sum_vol_tvn'] / pools['StockAlimentado']
            
            # Registrar estado de descarga de cada poza
            for p, d in pools.iterrows():
                records_desc.append({
                    'NumHora': hora,
                    'Poza': p,
                    'StockActual': d['Stock'],
                    'TVN_Actual': d['TVN_Actual'],
                    'StockAlimentado': d['StockAlimentado'],
                    'TVNAlimentado': d['TVNAlimentado'],
                    'Capacidad_Pozas': d['Capacidad']
                })
            
            # --- COCCIÓN (si corresponde) ---
            if hora >= cook_start_hour and cook_idx < len(speeds_fixed):
                speed = speeds_fixed[cook_idx]
                # Calcular TVN promedio ponderado
                total_stock = pools['Stock'].sum()
                avg_tvn = (pools['TVN_Actual'] * pools['Stock']).sum() / total_stock if total_stock > 0 else 0
                pools['Group'] = np.where(pools['TVN_Actual'] >= avg_tvn, 2, 1)
                order = pools.sort_values(['Group', 'TVN_Actual'], ascending=[True, False])
                
                rem = speed
                for p, row in order.iterrows():
                    if rem <= 0:
                        break
                    pre_stock = row['Stock']
                    cook_vol = min(pre_stock, rem)
                    if cook_vol <= 0:
                        continue
                    
                    tvn_pre = row['TVN_Actual']
                    # Reducir stock
                    pools.at[p, 'Stock'] = pre_stock - cook_vol
                    # Incrementar TVN en 1 unidad si queda stock
                    if pools.at[p, 'Stock'] > 0:
                        pools.at[p, 'TVN_Actual'] = tvn_pre + 1
                    tvn_post = pools.at[p, 'TVN_Actual']
                    
                    cook_records.append({
                        'Hora': hora,
                        'Poza': p,
                        'Group': int(row['Group']),
                        'TVN_PreCoccion': tvn_pre,
                        'TVN_PostCoccion': tvn_post,
                        'StockPreCoccion': pre_stock,
                        'VolCocido': cook_vol,
                        'StockPostCoccion': pre_stock - cook_vol,
                        'Velocidad': speed
                    })
                    rem -= cook_vol
                
                cook_idx += 1
            
            # --- ENVEJECIMIENTO POR HORA ---
            mask = pools['Stock'] > 0
            pools.loc[mask, 'TVN_Actual'] += 1
        return pd.DataFrame(records_desc), pd.DataFrame(cook_records)
    
    status_pozas_sim, df_coccion = sim_descarga_coccion(pools, intervals_rap, speeds_fixed, cook_start_hour, threshold_degradacion)
    
    
    df_coccion_grouped = df_coccion.groupby(["Hora"], as_index=False).agg(
        TVN_Pozas=('TVN_PreCoccion', 'mean'),
        TVN_IC=('TVN_PostCoccion', 'mean'),
        VolCocido=('VolCocido', 'sum'),
        VelocidadCoccion=('Velocidad', 'mean')
    )
    tvn_pozas = df_coccion_grouped["TVN_Pozas"].to_numpy()
    tvn_ingreso_cocina = df_coccion_grouped["TVN_IC"].to_numpy()
    velocidad_proceso = df_coccion_grouped["VelocidadCoccion"].to_numpy()
    
    
    def asignar_calidades(claves, valores, valor_comparativo):

        valores_finales = []
        for valor_comp in valor_comparativo:
            cont = 0
            for clave, valor in zip(claves, valores):
                if valor_comp <= clave:
                    cont += 1
                    valores_finales.append(valor)
                    break   
            if cont == 0:
                valores_finales.append('D')
        return valores_finales
    
    def asignar_receta(claves, valores, valor_comparativo):

        valores_finales = []
        max_valor = np.array(valores).max()
        for valor_comp in valor_comparativo:
            cont = 0
            for clave, valor in zip(claves, valores):
                if valor_comp <= clave:
                    cont += 1
                    valores_finales.append(valor)
                    break   
            if cont == 0:
                valores_finales.append(max_valor)
        return valores_finales
    
    def asignar_valores(claves, valores, valor_comparativo):

        valores_finales = []
        for valor_comp in valor_comparativo:
            for clave, valor in zip(claves, valores):
                if valor_comp == clave:
                    valores_finales.append(valor)
                    break   

        return np.array(valores_finales)
    
    
    calidades = asignar_calidades(UmbralTVN_Calidades[planta_selected].tolist(), UmbralTVN_Calidades["CALIDAD"].tolist(), tvn_ingreso_cocina)
    calidades = np.array(calidades)
    receta = asignar_receta(UmbralTVN_Calidades[planta_selected].tolist(), UmbralTVN_Calidades["DOSIFICACION"].tolist(), tvn_ingreso_cocina)
    
    # ——— 3. Parámetros de proceso ———
    fc_solidos            = intervals_rap["FCSolidosGrasayAguaBombeo"].values[0]
    # pct_solidos_licor     = 0.001
    pct_solidos_licor     = balance_ksa_df["%S Flujo de licor"].values[0]
    # pct_solidos_lodo      = 0.08
    pct_solidos_lodo      = balance_ksa_df["%S Flujo de lodo"].values[0]
    # pct_humedad_ksa       = 0.73
    pct_humedad_ksa       = balance_ksa_df["%H Flujo de KSA"].values[0]
    # pct_solidos_efluente  = 0.0004
    pct_solidos_efluente  = balance_ksa_df["%S Flujo de efluente"].values[0]
    # humedad_harina        = 7
    humedad_harina = asignar_valores(ValoresFijosPP["PROTEINA-HARINA"].tolist(), ValoresFijosPP["HumedadHarina"].tolist(), calidades)[0] * 100
    # grasa_h75             = 8
    # aportante_kp          = 24
    # aportante_ks          = 8
    # aportante_cc          = 14

    tvn_ponderado = resultados_pp_grouped.loc[resultados_pp_grouped["TVNPonderado"].notna(), "TVNPonderado"].values
    solidos_ab = (
        -445.24
        + tvn_ponderado * 294
        - 606.62 * grasa_h75
        +  88.804 * grasa_h75**2
    )
    grasa_ab = -47.169 + 0.943 * solidos_ab 

    # Pre‑cálculo de sólidos antes del clarificador (vector constante)
    solidos_ab_ic = (
        -445.24
        + tvn_pozas[:-2] * 294
        - 606.62 * grasa_h75
        +  88.804 * grasa_h75**2
    ) * fc_solidos

    n = velocidad_proceso.size
    EPS = 1e-8
    
    # def compute_protein(d):
    #     # Flujos iniciales
    #     flujo_de_ksa = velocidad_proceso * d
    #     base_seca    = flujo_de_ksa / 4
    #     flujo_ing    = base_seca * 1e6 / (solidos_ab_ic + EPS)

    #     # Parámetros intermedios
    #     pct_gr_ef    = pct_solidos_efluente / 2
    #     grasa_ab_ic  = (-47.169 + 0.943 * solidos_ab_ic) * fc_solidos
    #     pct_gr_ic    = grasa_ab_ic * 0.5

    #     # Flujos de lodo y efluente
    #     flujo_lodo   = flujo_ing * ((solidos_ab_ic * 1e-6) - pct_solidos_efluente) / (pct_solidos_lodo - pct_solidos_efluente + EPS)
    #     flujo_ef     = flujo_ing - flujo_lodo
    #     pct_gr_li    = pct_solidos_licor / 2
    #     pct_gr_ld    = (flujo_ing * pct_gr_ic / 1e6 - flujo_ef * pct_gr_ef) / (flujo_lodo + EPS)

    #     # Flujos de KSA y licor
    #     flujo_ksa    = flujo_lodo * (
    #                     ((1 - pct_gr_ld - pct_solidos_lodo)
    #                     - (1 - pct_gr_li - pct_solidos_licor))
    #                 / (pct_humedad_ksa - (1 - pct_gr_li - pct_solidos_lodo) + EPS)
    #                 )
    #     flujo_licor  = flujo_lodo - flujo_ksa
    #     pct_gr_ksa   = (flujo_lodo * pct_gr_ld - flujo_licor * pct_solidos_licor) / (flujo_ksa + EPS)

    #     proteina_ksa = 1 - humedad_ksa - pct_gr_ksa - ceniza_ksa
    #     # Composición integral (KK + dosificación)
    #     denom             = aportante_kp + aportante_ks + aportante_cc + d * 100 + EPS
    #     grasa_integral    = (aportante_kp * grasa_kk_prensa
    #                     + aportante_ks * grasa_kk_separadora
    #                     + aportante_cc * grasa_kk_cc
    #                     + d * pct_gr_ksa * 100) / denom
    #     # ceniza_integral   = (aportante_kp * ceniza_kk_prensa
    #     #                 + aportante_ks * ceniza_kk_separadora
    #     #                 + aportante_cc * ceniza_kk_cc
    #     #                 + d * ceniza_ksa * 100) / denom
    #     proteina_integral = (aportante_kp * proteina_kk_prensa
    #                     + aportante_ks * proteina_kk_separadora
    #                     + aportante_cc * proteina_kk_cc
    #                     + d * proteina_ksa * 100) / denom

    #     # Proteína de harina final
    #     proteina_harina = proteina_integral / (
    #         (grasa_integral + ceniza_integral + proteina_integral) / (100 - humedad_harina) + EPS
    #     )
    #     return proteina_harina
    
    
    
    return resultados_pp_grouped, volumen_ab, arranque, ValoresFijosPP, UmbralTVN_Calidades, velocidades_volumen_df, speeds_fixed, tvn_ingreso_cocina, tvn_pozas, calidades, intervals_rap


#### Ejecucion

In [4]:
cnxn_mio = pyodbc.connect('DRIVER={SQL Server};SERVER=srv-db-east-us003.database.windows.net;DATABASE=db_cfa_prd01;UID=userdbowner;PWD=$P4ssdbowner01#;')
id_pd = 7394
query_pd = f"""
SELECT A.*, B.id_planta, 
B.FlagPPyPZ,
case when TE = 'Propio C/Frio' then 'RC'
when TE = 'Propio S/Frio' then 'SF'
else 'SF' end as TIPO,
0.1 AS JuvenilesPonderado
FROM PlanDescargaOutput A
LEFT JOIN PlanDescargaModeloLog B
ON A.IDLog=B.id
WHERE B.FlagPPyPZ = 1 AND B.id = {id_pd};"""
discharge_plan_df_original = process_discharge_plan_data(pd.read_sql(query_pd, cnxn_mio))
discharge_plan_df = discharge_plan_df_original.copy()
planta_selected = discharge_plan_df["id_planta"].unique()[0]

# Hardcodeamos para poder consulttar datos completos
# planta_selected = "MALABRIGO"

C:\Users\MARAGON\AppData\Local\Temp\ipykernel_37752\557953416.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  discharge_plan_df_original = process_discharge_plan_data(pd.read_sql(query_pd, cnxn_mio))


In [5]:
print("=== SIMULACIÓN COMPLETA DEL BALANCE GENERAL ===")

# balance = BalanceGeneralVectorizadoCompleto()
id_prueba_pd = 7394
(datos_pd_mio, volumen_ab, arranque, ValoresFijosPP, 
 UmbralTVN_Calidades, velocidades_volumen_df, speeds_fixed,
 tvn_ingreso_cocina, tvn_pozas, calidades, intervals_rap) = cargar_datos_plan_descarga_y_tablas_mio(id_prueba_pd)
balance = BalanceGeneralVectorizadoCompleto(ValoresFijosPP, UmbralTVN_Calidades, speeds_fixed)

=== SIMULACIÓN COMPLETA DEL BALANCE GENERAL ===


C:\Users\MARAGON\AppData\Local\Temp\ipykernel_37752\3709967749.py:2069: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  discharge_plan_df_original = process_discharge_plan_data(pd.read_sql(query_pd, cnxn_mio))
C:\Users\MARAGON\AppData\Local\Temp\ipykernel_37752\3709967749.py:2096: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  grasa_df_original = pd.read_sql(query, cnxn_juv)
C:\Users\MARAGON\AppData\Local\Temp\ipykernel_37752\3709967749.py:2110: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  limpieza_tuberia_df_orig = pd.read_s

Grasa H75 5.45
Limite Base Seca 10.0
Limite Flujo IC 500


C:\Users\MARAGON\AppData\Local\Temp\ipykernel_37752\3709967749.py:233: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horas_dia = pd.date_range(
C:\Users\MARAGON\AppData\Local\Temp\ipykernel_37752\3709967749.py:239: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  horas_dia_poza = pd.date_range(
C:\Users\MARAGON\AppData\Local\Temp\ipykernel_37752\3709967749.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return (df[values] * df[weights]).sum() / df[weights].sum()
C:\Users\MARAGON\AppData\Local\Temp\ipykernel_37752\3709967749.py:67: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res

In [6]:
datos_pd_mio.shape

(54, 12)

In [7]:
index_arranque = datos_pd_mio[datos_pd_mio['flg_min'] == 1].iloc[0].name

In [8]:
if len(arranque) > 0:
    idx_min = (datos_pd_mio['Intervalo Poza'] - arranque.values[0]).abs().idxmin()
    cook_start_hour = datos_pd_mio.loc[idx_min]["NumHora"]
else:
    cook_start_hour = datos_pd_mio.loc[index_arranque, 'NumHora'] + 1

try:
    index_arranque = datos_pd_mio[datos_pd_mio['flg_min'] == 1].iloc[0].name
    hora_cook = datos_pd_mio.loc[index_arranque, 'NumHora'] + 1
    cook_start_hour_model = datos_pd_mio.loc[datos_pd_mio["NumHora"] == hora_cook, "Intervalo Poza"].values[0]
except:   
    cook_start_hour_model = None

In [9]:
try:
    index_arranque = datos_pd_mio[datos_pd_mio['flg_min'] == 1].iloc[0].name
    hora_cook = datos_pd_mio.loc[index_arranque, 'NumHora'] + 1
    cook_start_hour_model = datos_pd_mio.loc[datos_pd_mio["NumHora"] == hora_cook, "Intervalo Poza"].values[0]
except:   
    cook_start_hour_model = None

In [10]:
datos_pd_mio["RAPDescarga"].to_list()

[1.2381761978361667,
 1.2,
 1.206065184998636,
 1.2414857459943125,
 1.3023148236643411,
 1.3034758392292638,
 1.2,
 1.2535590003163555,
 1.3748583760453197,
 1.3,
 1.3,
 1.4816625310173697,
 1.4593227168249376,
 1.5129898246373674,
 1.509147085128872,
 1.5,
 1.4999999999999998,
 1.5,
 1.4999999999999998,
 1.5000000000000002,
 1.5,
 1.5000000000000002,
 1.5,
 1.4999999999999998,
 1.5,
 1.4999999999999998,
 1.5000000000000002,
 1.5,
 1.5,
 1.5000000000000002,
 1.5000000000000002,
 1.5,
 1.5,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [11]:
print(datos_pd_mio)
datos_iniciales = {
    'intervalo_chata': datos_pd_mio["Intervalo Chata"].to_numpy(dtype=object),
    'intervalo_poza':  datos_pd_mio["Intervalo Poza"].to_numpy(dtype=object),
    'descarga': datos_pd_mio["VolumenTotal"].to_numpy(dtype=object),
    'avance_descarga': datos_pd_mio["VolumenAcumulado"].to_numpy(dtype=object),
    
    'tvn_descarga': datos_pd_mio["TVNPonderado"].to_numpy(dtype=object),
    'tvn_ingreso_cocinas': tvn_ingreso_cocina,
    'tvn_pozas': tvn_pozas,
    'calidades': calidades,
}

balance.cargar_datos_completos_desde_excel(datos_iniciales)
resultados = balance.ejecutar_calculos_completos()

df_completo = balance.obtener_dataframe_completo()

       Intervalo Chata      Intervalo Poza  NumHora  VolumenTotal  \
0  2025-11-08 14:10:00 2025-11-08 14:50:00        1    161.750000   
1  2025-11-08 15:10:00 2025-11-08 15:50:00        2     87.601667   
2  2025-11-08 16:10:00 2025-11-08 16:50:00        3     79.415000   
3  2025-11-08 17:10:00 2025-11-08 17:50:00        4    240.283333   
4  2025-11-08 18:10:00 2025-11-08 18:50:00        5    174.383333   
5  2025-11-08 19:10:00 2025-11-08 19:50:00        6    221.433333   
6  2025-11-08 20:10:00 2025-11-08 20:50:00        7    218.566667   
7  2025-11-08 21:10:00 2025-11-08 21:50:00        8    105.366667   
8  2025-11-08 22:10:00 2025-11-08 22:50:00        9    247.133333   
9  2025-11-08 23:10:00 2025-11-08 23:50:00       10    199.183333   
10 2025-11-09 00:10:00 2025-11-09 00:50:00       11    202.650000   
11 2025-11-09 01:10:00 2025-11-09 01:50:00       12    167.916667   
12 2025-11-09 02:10:00 2025-11-09 02:50:00       13    182.001667   
13 2025-11-09 03:10:00 2025-11-09 

In [12]:
resultados

{'intervalo_chata': array([Timestamp('2025-11-08 14:10:00'), Timestamp('2025-11-08 15:10:00'),
        Timestamp('2025-11-08 16:10:00'), Timestamp('2025-11-08 17:10:00'),
        Timestamp('2025-11-08 18:10:00'), Timestamp('2025-11-08 19:10:00'),
        Timestamp('2025-11-08 20:10:00'), Timestamp('2025-11-08 21:10:00'),
        Timestamp('2025-11-08 22:10:00'), Timestamp('2025-11-08 23:10:00'),
        Timestamp('2025-11-09 00:10:00'), Timestamp('2025-11-09 01:10:00'),
        Timestamp('2025-11-09 02:10:00'), Timestamp('2025-11-09 03:10:00'),
        Timestamp('2025-11-09 04:10:00'), Timestamp('2025-11-09 05:10:00'),
        Timestamp('2025-11-09 06:10:00'), Timestamp('2025-11-09 07:10:00'),
        Timestamp('2025-11-09 08:10:00'), Timestamp('2025-11-09 09:10:00'),
        Timestamp('2025-11-09 10:10:00'), Timestamp('2025-11-09 11:10:00'),
        Timestamp('2025-11-09 12:10:00'), Timestamp('2025-11-09 13:10:00'),
        Timestamp('2025-11-09 14:10:00'), Timestamp('2025-11-09 15:10

In [13]:
print(f"\n✅ DataFrame generado: {df_completo.shape[0]} filas x {df_completo.shape[1]} columnas")

# Mostrar resultados clave
print("\n=== RESULTADOS CLAVE ===")
columnas_interes = ['intervalo_chata', 'velocidad', 'mp_ic_tn', 'solidos_ab', 'flujo_ksa_corregido', 'kk_integral_tn', 'har_ton']
columnas_disponibles = [col for col in columnas_interes if col in df_completo.columns]

# start_time = datetime.now()
now = datetime.now()
fecha_hora_actual = now.strftime('%Y-%m-%d_%H-%M-%S')
# time_elapsed = now - start_time
# print(f'Time elapsed (hh:mm:ss.ms) {time_elapsed}')


✅ DataFrame generado: 54 filas x 159 columnas

=== RESULTADOS CLAVE ===


In [16]:
if columnas_disponibles:
    # Mostrar solo filas con datos
    df_completo.to_excel(f"resultado_balance_general_{fecha_hora_actual}.xlsx", index=False)
    df_filtrado = df_completo[columnas_disponibles].dropna(how='all')
    print(df_filtrado.to_string())

       intervalo_chata  velocidad   mp_ic_tn    solidos_ab  flujo_ksa_corregido  kk_integral_tn  har_ton
0  2025-11-08 14:10:00        NaN        NaN           NaN                  NaN             NaN      NaN
1  2025-11-08 15:10:00        NaN        NaN           NaN                  NaN             NaN      NaN
2  2025-11-08 16:10:00     140.00  145.65600   3951.479047                  8.4       145.65600      NaN
3  2025-11-08 17:10:00     140.00  145.65600   4547.383646                  8.4       145.65600      NaN
4  2025-11-08 18:10:00     150.00  156.06000   3817.232403                  9.0       156.06000      NaN
5  2025-11-08 19:10:00     180.00  187.27200   3978.244420                 10.8       187.27200      NaN
6  2025-11-08 20:10:00     180.00  187.27200   4368.028974                 10.8       187.27200      NaN
7  2025-11-08 21:10:00     180.00  187.27200   4731.095097                 18.0       187.27200      NaN
8  2025-11-08 22:10:00     180.00  187.27200   5348.974

In [17]:
balance.data

{'intervalo_chata': array([Timestamp('2025-11-08 14:10:00'), Timestamp('2025-11-08 15:10:00'),
        Timestamp('2025-11-08 16:10:00'), Timestamp('2025-11-08 17:10:00'),
        Timestamp('2025-11-08 18:10:00'), Timestamp('2025-11-08 19:10:00'),
        Timestamp('2025-11-08 20:10:00'), Timestamp('2025-11-08 21:10:00'),
        Timestamp('2025-11-08 22:10:00'), Timestamp('2025-11-08 23:10:00'),
        Timestamp('2025-11-09 00:10:00'), Timestamp('2025-11-09 01:10:00'),
        Timestamp('2025-11-09 02:10:00'), Timestamp('2025-11-09 03:10:00'),
        Timestamp('2025-11-09 04:10:00'), Timestamp('2025-11-09 05:10:00'),
        Timestamp('2025-11-09 06:10:00'), Timestamp('2025-11-09 07:10:00'),
        Timestamp('2025-11-09 08:10:00'), Timestamp('2025-11-09 09:10:00'),
        Timestamp('2025-11-09 10:10:00'), Timestamp('2025-11-09 11:10:00'),
        Timestamp('2025-11-09 12:10:00'), Timestamp('2025-11-09 13:10:00'),
        Timestamp('2025-11-09 14:10:00'), Timestamp('2025-11-09 15:10

In [18]:
velocidades_volumen_df.columns

Index(['id', 'PLANTA', 'GRASA MP', 'VOLUMEN MIN ARRANQUE', 'RANGO_MP_TM',
       'MIN - TM', 'MAX - TM', 'VELOCIDAD DE ARRANQUE', 'VELOCIDAD MAXIMA',
       'VELOCIDAD DE CIERRE', 'VEL_INTERMEDIA1', 'VEL_INTERMEDIA2'],
      dtype='object')

In [19]:
# 🔹 Materia Prima - Pescado
PESCADO_HUMEDAD = {'A': 75, 'B': 74, 'C': 73, 'D': 72}
PESCADO_CENIZAS = {'A': 4.0, 'B': 4.2, 'C': 4.4, 'D': 4.6}

In [20]:
velocidades_volumen_df

,id,PLANTA,GRASA MP,VOLUMEN MIN ARRANQUE,RANGO_MP_TM,MIN - TM,MAX - TM,VELOCIDAD DE ARRANQUE,VELOCIDAD MAXIMA,VELOCIDAD DE CIERRE,VEL_INTERMEDIA1,VEL_INTERMEDIA2
0,1,MALABRIGO,<=5,900,>= 2500,3000,50000,120,200,80,150,120
1,2,MALABRIGO,<=5,900,2000 a 2500,2000,3000,120,180,80,150,120
2,3,MALABRIGO,<=5,900,1500 a 2000,1500,2000,120,170,80,140,110
3,4,MALABRIGO,<=5,900,1000 a 1500,1000,1500,120,150,80,135,110
4,5,MALABRIGO,<=5,900,750 -1000,750,1000,120,150,80,135,110
5,6,MALABRIGO,<=5,900,500 a 750,500,750,80,120,70,100,90
6,7,MALABRIGO,<=5,900,250 a 500,250,500,60,70,60,70,70
7,8,MALABRIGO,<=5,900,<= 250,0,250,35,40,35,40,40
8,9,MALABRIGO,>5,900,>= 2500,3000,50000,140,180,80,150,120
9,10,MALABRIGO,>5,900,2000 a 2500,2000,3000,140,170,80,150,100


In [21]:
speeds_fixed

[np.float64(140.0),
 np.float64(140.0),
 np.float64(150.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(180.0),
 np.float64(120.0),
 np.float64(80.0),
 np.float64(55.149999999999636)]